In [3]:
import numpy as np
import pandas as pd
import  re
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq
from Bio.SeqUtils.MeltingTemp import make_table, DNA_NN2
import pyperclip
import importlib
import DNA_tools as dt
#https://biopython.org/docs/1.75/api/Bio.SeqUtils.MeltingTemp.html
Breslauer= mt.DNA_NN1
Santalucia= mt.DNA_NN4 #same as benchling
Tm_Opt =Santalucia

## This notebook intends to automatically design primers with overhangs for USER cloning

In [4]:
#Design User overhang
HM='A'
for i in range(7):
    r=np.random.rand(1)
    if r <=0.625:
        HM+=np.random.choice(['G','C'])
    else:
        HM+=np.random.choice(['A','T'])
HM+='U'
print(HM, '  GC%:',(HM.count('C')+HM.count('G'))/len(HM))



Tm=0
for bp in HM:
    if bp == 'A' or bp == 'T':
        Tm+=2
    else:
        Tm+=4
Rv=''
for bp in HM[::-1]:
    if bp == 'A':
        Rv+='T'
    if bp == 'T' or bp == 'U':
        Rv+='A'
    if bp == 'C':
        Rv+='G'
    if bp == 'G':
        Rv+='C'
        
        
print('TM:',Tm, len(HM))
print('Reverse: ', Rv)

ACGGGGATU   GC%: 0.5555555555555556
TM: 30 9
Reverse:  AATCCCCGT


In [5]:
#Design 50bp random overhang for yeast HR:


continue_search = True
while continue_search == True:
    Overhang=''
    for i in range(50):
        r=np.random.rand(1)
        
        bp=''
        if r<= 0.25:
            bp='A'
        elif r>0.25 and r<=0.5:
            bp='T'
        elif r>0.5 and r<=0.75:
            bp='C'
        else:
            bp='G'
        Overhang+=bp
    
    GC =(Overhang.count('C')+Overhang.count('G'))/len(Overhang)
    if GC >= 0.475 and GC <= 0.525: #makes sure the overhang is balanced
        continue_search = False
        
Rv_Overhang= dt.reverse_complement(Overhang)
print('GC content:', GC, '\n','Overhang Fwd: ', Overhang,'\n Overhang Rv: ', Rv_Overhang)

GC content: 0.48 
 Overhang Fwd:  TGGTCCGTGACTTGCATCGGTCATAGGGAGTTCTGTTTAATTGGTGACTC 
 Overhang Rv:  GAGTCACCAATTAAACAGAACTCCCTATGACCGATGCAAGTCACGGACCA


### Define your user overhangs:

In [6]:
#forward seq of user overhangs:
User_Overhangs={}
User_Overhangs['GV1']= 'CGTGCGAu'
User_Overhangs['GV2']= 'ATCGCGTG'#'CACGCGAu'
User_Overhangs['U1'] = 'AGCCCTCTu'
User_Overhangs['U2'] = 'ACACAGGCu'
User_Overhangs['U3'] = 'ATGCCACCu'
User_Overhangs['U4'] = 'AAGGAGCGu'
User_Overhangs['U5'] = 'ACCGTAGCu'
User_Overhangs['U6'] = 'ACTGCGAGu'
User_Overhangs['U7'] = 'AGTGCAGGu'
User_Overhangs['none'] = 'TATATATATATATA'

Copy and paste sequence from benchling 
The primers will be designed taking the immediate bp at the start and end of sequence 
(bp are added to annealling sequence till Tm reaches specified Tm (or avg Tm if speficied primers))
Always copy sequence as is (not reverse complement). 
If the seq needs to be reversed in assembly set partX['need_reverse']=1
If you provide primers, input annealing sequence from 5'>>3'
Specify the user sites as wanted in the final construct

In [11]:
#Imput your parts here
part1={}
part1['name']='pMeLS0025(pCYC_benO::yEGFP::Cyc1t)'
part1['seq']='gatccaggcaactttagtgctgacacataatactccataggtattttattatacaaataatgtgtttgaacttattaaaacattcttttaaggtataaacaacaggcatatatatatgtgtgcgacgacacatgatcatatggcatgcatgtgctctgtatgtatataaaactcttgttttcttcttttctctaaatattctttccttatacattaggacctttgcagcataaattactatacttctatagacacacaaacacaaatacacacactaaattaataatctgtcataaaacaatgtctaaaggtgaagaattattcactggtgttgtcccaattttggttgaattagatggtgatgttaatggtcacaaattttctgtctccggtgaaggtgaaggtgatgctacttacggtaaattgaccttaaaatttatttgtactactggtaaattgccagttccatggccaaccttagtcactactttcggttatggtgttcaatgttttgcgagatacccagatcatatgaaacaacatgactttttcaagtctgccatgccagaaggttatgttcaagaaagaactatttttttcaaagatgacggtaactacaagaccagagctgaagtcaagtttgaaggtgataccttagttaatagaatcgaattaaaaggtattgattttaaagaagatggtaacattttaggtcacaaattggaatacaactataactctcacaatgtttacatcatggctgacaaacaaaagaatggtatcaaagttaacttcaaaattagacacaacattgaagatggttctgttcaattagctgaccattatcaacaaaatactccaattggtgatggtccagtcttgttaccagacaaccattacttatccactcaatctgccttatccaaagatccaaacgaaaagagagaccacatggtcttgttagaatttgttactgctgctggtattacccatggtatggatgaattgtacaaataaatcgcgtgcattcatccgctctaaccgaaaaggaaggagttagacaacctgaagtctaggtccctatttatttttttatagttatgttagtattaagaacgttatttatatttcaaatttttcttttttttctgtacagacgcgtgtacgcatgtaacattatactgaaaaccttgcttgagaaggttttgggacgctcgaagatcgcgtcagctgaa' #sequence to clone
part1['need_reverse']=0 #if parts needs to be reversed:1 else:0
part1['Primer_up_U']='GV1' #user site up
part1['Primer_down_U']='GV2' #user site down
part1['Primer_up']=''
part1['Primer_up_Tm']=''
part1['Primer_down']=''
part1['Primer_down_Tm']=''

part2={}
part2['name']='pMeLS0076(Rev1p::BenM(H110R, F211V, Y286N))'
part2['seq']='gaatgcggccgctttaccaatttggtggttcagtaaaaccttcgtaggcgtagatttgtctgatggtttcgtataatgagtagatgtaggtggattcttccatgtttctaacagcaatgtagattggggtaatagcatctggatctaataatgggacgtaggacaagttgaataattgaatggattgggtagaagctggaaccaatgaaataccttcaccagcagcaaccaaacccaaggccaattggacttctctaacttcgttaatcttggtaggttccaaaccatggtcagagaagatgttcataacatgagtagagaagtttggctttggagaagatgggtacaacaagatcttttcgtcgatcaaatcgttcaagtgaacacccttatccttcatttgattcaatggatgggaagcatgaacagcaaccatcaatctttcgtttctcaacaaggttctcttgatggctggatcagaaatcttcaatctaccaaaaccagcgtcaattctaccttctttcaaagcttcggtttgagccttagtacccatttcgtataattcgattctcaagtttggatgagcttgtctgtacaagtggataattcttggcaacaaaccgaacaacaaggaaccaacaaaaccgattctaatggtcttttcaacagaggcaattctcttggtcatactgaccatttgatcaacgttgcttagcaacttgatggcgtattggtaaaagaaatgaccttctggagtagtcttaactggtctggaacctctttccaataattggatacccaattcttcttccaagttttggatttgtctggataatggtggttgagcaatacacaacttgtcggcagcctttgtaaaagattgttcttcaacaacggcaacgaagtatctcaagtgtctcaattccattgttttatgacagatcgctggatatgcctagaaatgcagtatcgatttattttgagaaaatctgtcaactcgttacttctttgatagggaaatctttagttaaatttgtttaaaattaagaaccaggtctttagactatgccaaaaggtattggtgcttaaaggttgccgtaaactctaactctatatatctctaagccaatgtttaagttgatgcgatgtgcatgatcacttcacatttttttcatgattttgttttcttccttagctacttttttccgtcttggtctgtgtaaaaaaaaaatcgcttcactgaagaaaaatataaataaaaaaaggacatcagcaagttcgtagtcaataaatgtgatgagtaatttgcgataatctaccccaactcaaagcattcaattcaaatgaaagaagagttcgtgaattttaatctggcttacagattggatcatagcttctaatatatcttcctttttattttaaagtgattaaaatatgtggatgtctgcattaaggtgaaggcagactatagtgaaaaatagtagcaaaggcttcatataatgaactacctagttgtattcaattggaatctgtgaacgcggcattaaaagaatatttgagctttttttgattgtattaacctgaagctatcacggtttagactcattaaaagaaaaatatccggttgtatcggataacaacaccgctgtagtagcacagttttttgatcatagtttcattatttttttaaatatgcttgtatacagtaatctagttttggacggattattcatgtcagtaagattttactatctactttaaaattatcgaaacagatatcaactatacatccactcattattggcatagcgaaataaaatttatgaataggaaggatataaaacgaaaattgaaatcgtttagtattatatgaaactattagtcttttcgttttcgtcgtcatacttacattcttcttttataaatattgttgtgcctaagaagagctccagcttttgttccctttagtgagggttaattgcgcgcttggcgtaatcatggtcatagctgtttcctgtgtgaaattgttatccgct'
part2['need_reverse']=0 #if parts needs to be reversed:1 else:0
part2['Primer_up_U']='GV1' #user site up
part2['Primer_down_U']='GV2' #user site down
part2['Primer_up']=''
part2['Primer_up_Tm']=''
part2['Primer_down']=''
part2['Primer_down_Tm']=''

part2B={}
part2B['name']='pMeLS0076(Rev1p::BenM_MP17_D08)'
part2B['seq']='Ttaccaatttggtggttcagtaaaaccttcgtaggcgtagatttgtctgatggtttcgtataatgagtagatgtaggtggattcttccatgtttctaacagcaatgtagattggggtaatagcatctggatctaataatgggacgtaggacaagttgaataattgaatggattgggtagaagctggaaccaatgaaataccttcaccagcagcaaccaaacccaaggccaattggacttctctaacttcgttaatcttggtaggttccaaaccatggtcagagaagatgttcataacatgagtagagaagtttggctttggagaagatgggtacaacaagatcttttcgtcgatcaaatcgttcaagtgaacacccttatccttcatttgattcaatggatgggaagcatgaacagcaaccatcaatctttcgtttctcaacaaggttctcttgatggctggatcagaaatcttcaatctaccaaaaccagcgtcaattctaccttctttcaaagcttcggtttgagccttagtacccatttcgtataattcgattctcaagtttggatgagcttgtctgtacaagtggataattcttggcaacaaaccgaacaacaaggaaccaacaaaaccgattctaatggtcttttcaacagaggcaattctcttggtcatactgaccatttgatcaacgttgcttagcaacttgatggcgtattggtaaaagaaatgaccttctggagtagtcttaactggtctggaacctctttccaataattggatacccaattcttcttccaagttttggatttgtctggataatggtggttgagcaatacacaacttgtcggcagcctttgtaaaagattgttcttcaacaacggcaacgaagtatctcaagtgtctcaattccattgttttatgacagatcgctggatatgcctagaaatgcagtatcgatttattttgagaaaatctgtcaactcgttacttctttgatagggaaatctttagttaaatttgtttaaaattaagaaccaggtctttagactatgccaaaaggtattggtgcttaaaggttgccgtaaactctaactctatatatctctaagccaatgtttaagttgatgcgatgtgcatgatcacttcacatttttttcatgattttgttttcttccttagctacttttttccgtcttggtctgtgtaaaaaaaaaatcgcttcactgaagaaaaatataaataaaaaaaggacatcagcaagttcgtagtcaataaatgtgatgagtaatttgcgataatctaccccaactcaaagcattcaattcaaatgaaagaagagttcgtgaattttaatctggcttacagattggatcatagcttctaatatatcttcctttttattttaaagtgattaaaatatgtggatgtctgcattaaggtgaaggcagactatagtgaaaaatagtagcaaaggcttcatataatgaactacctagttgtattcaattggaatctgtgaacgcggcattaaaagaatatttgagctttttttgattgtattaacctgaagctatcacggtttagactcattaaaagaaaaatatccggttgtatcggataacaacaccgctgtagtagcacagttttttgatcatagtttcattatttttttaaatatgcttgtatacagtaatctagttttggacggattattcatgtcagtaagattttactatctactttaaaattatcgaaacagatatcaactatacatccactcattattggcatagcgaaataaaatttatgaataggaaggatataaaacgaaaattgaaatcgtttagtattatatgaaactattagtcttttcgttttcgtcgtcatacttacattcttcttttataaatattgttgtgcctaagaa'
part2B['need_reverse']=0 #if parts needs to be reversed:1 else:0
part2B['Primer_up_U']='GV1' #user site up
part2B['Primer_down_U']='GV2' #user site down
part2B['Primer_up']=''
part2B['Primer_up_Tm']=''
part2B['Primer_down']=''
part2B['Primer_down_Tm']=''


part3={}
part3['name']='pTDH2'
part3['seq']='tttgttttgtttgtttgtgtgatgaatttaatttgaaactaagttcttggtttttttaacatttaaacaaacaataataaaatgaatagacacacaagaaaaaaactatcttgaatgagtgcaacctttatatattcttccaaacctacaaaaaaattgcaggtctgcgatcactgggtactatcaaaacacatattagcatacgtatgtgttcgtacgtgcggtccgcaaggtctttgtgacagccatgacatgattggattgaatagttcacttcagctgtcaccaccagttttaggccctaacacatcagtcgagttccactaggctttgttagccgtagctgcgttgcaacaactgcgcaaaacgcgcgacttcctgtaaatgccgcagatcacgtggtaaagaaaagaaacaacaggccgacccagtttcgttgtttcttttcattatttctggatgctcttcacaacgtatttcatgacatacttattgaattagtggaagcccaaaacatcattaccctggaaaacttaattgacgtaaattaacttcagttgtcttatgttttttgcatccttaactttagccgcgcagggataatcatatatgtcgaaagagcaactaaacaatacttaaaaacgtgtttgaacttttacgaaagggtaacgacaaataatcattcactggaaaaaccaataaaccataagtaaatgaaaaactgtttcaaccgatgttggtactttccatgtcttaaaagagtacaatttataaattaatgaaaacacagaagtatttagatcggctcaaatgtttttggacattaaaagatcttgaaactgagtaagatgctcagaatacccgtcaagataagagtataatgtagagtaatataccaagtattcagcatattctcctcttcttttgtataaatcacggaagggatgatttataagaaaaatgaatactattacacttcatttaccaccctctgatctagat'
part3['need_reverse']=1 #if parts needs to be reversed:1 else:0
part3['Primer_up_U']='GV1' #user site up
part3['Primer_down_U']='U1' #user site down
part3['Primer_up']=''
part3['Primer_up_Tm']=''
part3['Primer_down']='aaatctagatcagagggtggtaaat'
part3['Primer_down_Tm']=''

part4={}
part4['name']='pCFB1237_ScTkl1'
part4['seq']='tcagaaagcttttttcaaaggagaaattagcttgtcacccttatagaatgcaatggtcttttgagctctttcagcaacaccttctggggtgaaaccgaagaacttgaagacttctggtgccttaccggaggcaccaaatctgtcaataccgaaggattgatgagcgtatttgccccaacatgtggtagccaaaacttcaacagacatgattggaacgttgtctggtaagactgatagtctgtattctaggggttgtttgtcaaaagtgaagaaatctggtagagaaacaacacgagccttgatgttctttgcggccaaagtcttagcagcttcaacactcaaagacacttcggaaccagtagccactaaaataatatctgggttagcaacatcttgtagtacgtaaccacccttagaagcgctttcaatagagctaccttccaattgtggcaagttttgtctggacaaagcaatgatacttggagtatgcttggattctaaagagttcttgtaggcggcagaaacttcgttaccatcagctggtctccaaacttgaatgtttggtagggatctgaagtgtgctaaagtttcaataggttgatgtgttggaccatcttcaccgacaccgatagagtcatgtgtagcaacccaaataactgggtggccagacaaagcggacaatctaacggcaccagcagcataagaaacgaagttcaagaaagtaccaccgtatggtttgtagttggcaccgaaagctgaaataccgttcattatggcacccatagcgtgttctctaataccgtacctaatgtatctaccagagtagttacctgaaccggaagaaggaggttggaagtcaagggcttccttccatctggtcaagttagaaggtgttaaatcggcagaaccaccaatcaactctggcaattgattgtaaacatcctcaagaacagtttctgataattttctagtggccacggcagagtccttggcggtgtaagttggcaacttagattcccaatttgcgggtagttggccgctcaatcttctagccaattcagcacctaattctgggaatttcttttggtattcgctgaacaacttgttccacttgttgttggcttcgacacctggctttaaaattgtcttttggtagtggtcgtaaacttcttgtggaacaacaaaggacttgtctgggttgaaaccgaatttgctctttagttgtttaacatcatctgctttcaatggggcaccgtgcacagagtgagagccggcatgcaaggaaccgtaaccaatggttgtggtcattttgatcaaagttggtttgtccttggataacttagcttgagcaatagccttggcaataccggctagatcttcgttaccattttctacgtacaaaacttcccaaccgtaggcttcgtatctcttagcaacatcttcatcgaatgagatactggtagcaccatcgatagtgatcttgttgtcatcgtagatggcaatcaagttacccaatttcaaatgaccagccaaggaggaagcttctgaagaaataccttcttgcaaacaaccgtcacccaagaaaacataggtgtagttgtcagacaaggtaaagcccggcttgttgtaagtggcagccaggttagcttgagccatggccataccaacagcgttggagataccttgacctaatggaccggtagtaacttcaacacctggcaactcaaattcaggatgacctggtgttctggaacccaactgtctgaactgtttcaagtcttcaatagacagatcgtaaccagtcaaatgtagcatagaatacaacaaagcgaccgcgtgaccgttagacaagacaaatctatctctgttgatccagtctgggttggttgggttcatgcgcatttgactccatagaacgtgtgcagctggtgccatacccaatggagcacctgggtgacctgagttggccttggatacggtgtccacagccaaaattcttatggtggagacggctagcttatcaatgtcagtgaattgagtcattgt'
part4['need_reverse']= 1 #if parts needs to be reversed:1 else:0
part4['Primer_up_U']='U1' #user site up
part4['Primer_down_U']='U2' #user site down
part4['Primer_up']=''
part4['Primer_up_Tm']=''
part4['Primer_down']=''
part4['Primer_down_Tm']=''
part5={}
part5['name']='ANTE113_Tdh3bidir'
part5['seq']='TCTGCAGGTAGGGAAAGATATGAGCTATACAGCGGAATTTCCATATCACTCAGATTTTGTTATCTAATTTTTTCCTTCCCACGTCCGCGGGAATCTGTGTATATTACTGCATCTAGATATATGTTATCTTATCTTGGCGCGTACATTTAATTTTCAACGTATTCTATAAGAAATTGCGGGAGTTTTTTTCATGTAGATGATACTGACTGCACGCAAATATAGGCATGATTTATAGGCATGATTTGATGGCTGTACCGATAGGAACGCTAAGAGTAACTTCAGAATCGTTATCCTGGCGGAAAAAATTCATTTGTAAACTTTAAAAAAAAAAGCCAATATCCCCAAAATTATTAAGAGCGCCTCCATTATTAACTAAAATTTCACTCAGCATCCACAATGTATCAGGTATCTACTACAGATATTACATGTGGCGAAAAAGACAAGAACAATGCAATAGCGCATCAAGAAAAAACACAAAGCTTTCAATCAATGAATCGAAAATGTCATTAAAATAGTATATAAATTGAAACTAAGTCATAAAGCTATAAAAAGAAAATTTATTTAAATGCAAGATTTAAAGTAAATTCAC'
part5['need_reverse']= 1 #if parts needs to be reversed:1 else:0
part5['Primer_up_U']='U2' #user site up
part5['Primer_down_U']='GV2' #user site down
part5['Primer_up']=''
part5['Primer_up_Tm']=''
part5['Primer_down']=''
part5['Primer_down_Tm']=''


User1=[part1]
User2=[part2]
User2B=[part2B] #pCCM010
User3=[part3, part4,part5]

In [12]:
part6={}
part6['name']='FBA1p'
part6['seq']='tttgaatatgtattacttggttatggttatatatgacaaaagaaaaagaagaacagaagaataacgcaaggaagaacaataactgaaattgatagagaagtattatgtctttgtctttttataataaatcaagtgcagaaatccgttagacaacatgagggataaaatttaacgtgggcgaagaagaaggaaaaaagtttttgtgagggcgtaattgaagcgatctgttgattgtagattttttttttttgaggagtcaaagtcagaagagaacagacaaatggtattaaccatccaatacttttttggagcaacgctaagctcatgcttttccattggttacgtgctcagttgttagatatggaaagagaggatgctcacggcagcgtgactccaattgagcccgaaagagaggatgccacgttttcccgacggctgctagaatggaaaaaggaaaaatagaagaatcccattcctatcattatttacgtaatgacccacacatttttgagattttcaactattacgtattacgataatcctgctgtcattatcattattatctatatcgacgtatgcaacgtatgtgaagccaagtaggcaattatttagtactgtcagtattgttattcatttcttagggcttgcgtcagcttgatgaggatttgtgatagtagccgttggaggcatgaagggcattggaaacatggggaaaggaggtataccgaaaggtaacggtactggtggcactggcgtaccgccgttatttacagaagttggaaggctggtattgttgttcaagccagcggtgccagttggatccatcaacttcattttcttagtttctggctggttgccatctttagagcttccgtgtagttcctcttgtttcttcaaaaacgtttcgacctccttttccttgtcctggtcgggtactaaagaatcgagaaggatatcgcgggtttcgcaattggggcatacaaagtcgctctctaccagt'
part6['need_reverse']=1 #if parts needs to be reversed:1 else:0
part6['Primer_up_U']='GV1' #user site up
part6['Primer_down_U']='U1' #user site down
part6['Primer_up']='tttgaatatgtattacttggttatg'
part6['Primer_up_Tm']=''
part6['Primer_down']='actggtagagagcgacttt'
part6['Primer_down_Tm']=''

part7={}
part7['name']='pCFB1239_PaAroZ'
part7['seq']='tcacaaagcagctgacaaagacaacattctgataggttgttgtgtagcttgttgttgagttggagagtcaacggtgatcttcatatctctaaccaatttagcccaagaaacagcacctcttctagctaattcttctggaacaccaaaaccggtatcggacattcttctgttgaataattctaaggaaacccaaccttcaaaacccaaaccgttgaaaaaagcccaagcaatttcttttactggcaagtaagcacctctatccttttcaccgtagaataatctgcagtttctagaccaagacattctagctggttgttctggatcatagaatctgtgacctggaaccaatggtttcttcaacttttcggcatcaacaacttgaacgtagaaaaccttggaaacgtcaactctttcaaccaatctggcaatggactttctgatagcttcttcagcatttggagttctaccagaagcaacagttggatcagcataaactctaccggcaatgttaaaagtgtccaaacaaacaccgaagtttggtctgttaactctttgaacaacttcccaagatctttcccaagtatcaactctagtggaccaacacaaagcttcgtaaacgaatctaattggtgggttagcttgtaaacccatatcagctacttcttgcaagtcggagacaatcaaagagatatcttcagtaacttcttcggctggcaagaagttggctggaatttgaataatatcggtatccaattcatgggccaattcgatccagaattccaattgttccaatcttctttcgtgttcttctctatccaacaaaccatcgtattgagagaatggttgcaagcaaacgatttcgatgtttctgacttggcacattctcaaaatttgtctagcagcagacaattgagcagctggtgatggaccacatggagatggagtttcaccagacaatctataggcgacatcagccaaatcttcatggaataattccaaaccttggtaaccgtactttctagccatatccaacttagtagtgaaagaatgaccggcataacatctacccaaagacatagaggtaatggccaacttggatggcattgt'
part7['need_reverse']= 1 #if parts needs to be reversed:1 else:0
part7['Primer_up_U']='U1' #user site up
part7['Primer_down_U']='U2' #user site down
part7['Primer_up']=''
part7['Primer_up_Tm']=''
part7['Primer_down']=''
part7['Primer_down_Tm']=''

part8={}
part8['name']='His5t'
part8['seq']='atagattaatttaaacagtatatgtacagttttatatatatatatatatatatatacatatataaagaaacctgtgcgttttttgtattttcaaatacatttagttttggcgccacttctataaaaggtctcatgatattgttac'
part8['need_reverse']= 0  #if parts needs to be reversed:1 else:0
part8['Primer_up_U']='U2' #user site up
part8['Primer_down_U']='GV2' #user site down
part8['Primer_up']='atagattaatttaaacagtatatgtacagttt'
part8['Primer_up_Tm']=''
part8['Primer_down']='gtaacaatatcatgagaccttttata'
part8['Primer_down_Tm']=''

part9={}
part9['name']='TPI1p'
part9['seq']='ttttagtttatgtatgtgttttttgtagttatagatttaagcaagaaaagaatacaaacaaaaaattgaaaaagattgatttagaattaaaaagaaaaatatttacgtaagaagggaaaatagtaaatgttgcaagttcactaaactcctaaattatgctgccctttatattccctgttacagcagccgagccaaaggtatataggctcctttgcattagcatgcgtaacaaaccacctgtcagtttcaaccgaggtggtatccgagagaattgtgtgattgctttaattaatttcggagaatctcacatgccactgaagattaaaaactggatgccagaaaaggggtgtccaggtgtaacatcaatagaggaagctgaaaagtcttagaacgggtaatcttccaccaacctgatgggttcctagatataatctcgaagggaataagtagggtgataccgcagaagtgtctgaatgtattaaggtcctcacagtttaaatcccgctcacactaacgtaggattattataactcaaaaaaatggcattattctaagtaagttaaatatccgtaatctttaaacactatgtagttaggtctccctcacaatcagtccatttgggtagcacggtcctcaggacgtatctattgatggattcgtccagttccatcaccattacgctcccgttaggaacattggtaaacgattcaaactcttcgtatgtccatctaaaccatttcatcaggaatactctggaataaataccatgtgtaactaggacaacaacatctctgggtcttctctcttgcctatcatggaagtgcctgaataaagtctcttggaaactggcgactctgtcatatacatctgccgcactttctccatgagggaatctgaagaagaaatgaccatacgtagatctcttcttcataacgtcctgcatgctattgattttttggaaattaccaaaatcttgttcccttattcttggctcatcctt'
part9['need_reverse']=1 #if parts needs to be reversed:1 else:0
part9['Primer_up_U']='GV1' #user site up
part9['Primer_down_U']='U1' #user site down
part9['Primer_up']=''
part9['Primer_up_Tm']=''
part9['Primer_down']=''
part9['Primer_down_Tm']=''

part10={}
part10['name']='pCfB2193_NatMX'
part10['seq']='accatgggtaccactcttgacgacacggcttaccggtaccgcaccagtgtcccgggggacgccgaggccatcgaggcactggatgggtccttcaccaccgacaccgttttccgcgtcaccgccaccggggacggcttcaccctgcgggaggtgccggtggacccgcccctgaccaaggtgttccccgacgacgaatcggacgacgaatcggacgacggggaggacggcgacccggactcccggacgttcgtcgcgtacggggacgacggcgacctggcgggcttcgtggtcgtctcgtactccggctggaaccgccggctgaccgtcgaggacatcgaggtcgccccggagcaccgggggcacggggtcgggcgcgcgttgatggggctcgcgacggagttcgcccgcgagcggggcgccgggcacctctggctggaggtcaccaacgtcaacgcaccggcgatccacgcgtaccggcggatggggttcaccctctgcggcctggacaccgccctgtacgacggcaccgcctcggacggcgagcaggcgctctacatgagcatgccctgcccctaa'
part10['need_reverse']= 0#if parts needs to be reversed:1 else:0
part10['Primer_up_U']='U1' #user site up
part10['Primer_down_U']='U2' #user site down
part10['Primer_up']=''
part10['Primer_up_Tm']=''
part10['Primer_down']=''
part10['Primer_down_Tm']=''

part11={}
part11['name']='LSC2t'
part11['seq']='aaaattaaaaaaaaaaaaaagaaattttttccaaaagagtatatttatatgtatgtatacatgtagggaaaataagaaactttattaatagtaaaaaagcatatatactttattattaactcttttgtttttctcgagaagc'
part11['need_reverse']=1 #if parts needs to be reversed:1 else:0
part11['Primer_up_U']='U2' #user site up
part11['Primer_down_U']='GV2' #user site down
part11['Primer_up']='aaaattaaaaaaaaaaaaaagaaattttttcc'
part11['Primer_up_Tm']=''
part11['Primer_down']='gcttctcgagaaaaacaaaagagtta'
part11['Primer_down_Tm']=''

part12={}
part12['name']='pCYC1'
part12['seq']='cagcattttcaaaggtgtgttcttcgtcagacatgttttagtgtgtgaatgaaataggtgtatgttttctttttgctagacaataattaggaacaaggtaagggaactaaagtgtagaataagattaaaaaagaagaacaagttgaaaaggcaagttgaaatttcaagaaaaaagtcaattgaagtacagtaaattgacctgaatatatctgagttccgacaacaatgagtttaccaaagagaacaatggaataggaaactttgaacgaagaaaggaaagcaggaaaggaaaaaatttttaggctcgagaacaatagggcgaaaaaacaggcaacgaacgaacaatggaaaaacgaaaaaaaaaaaaaaaaacacagaaaagaatgcagaaagatgtcaactgaaaaaaaaaaaggtgaacacaggaaaaaaaataaaaaaaaaaaaaaaaaaaggaggacgaaacaaaaaagtgaaaaaaaatgaaaatttttttggaaaaccaagaaatgaattatatttccgtgtgagacgacatcgtcgaatatgattcagggtaacagtattgatgtaatcaatttcctacctgaatctaaaattcccgggagcaagatcaagatgttttcaccgatctttccggtctctttggccggggtttacggacgatggcagaagaccaaagcgccagttcatttggcgagcgttggttggtggatcaagcccacgcgtaggcaatcctcgagcagatccgccaggcgtgtatatatagcgtggatggccaggcaactttagtgctgacacatacaggcatatatatatgtgtgcgacgacacatgatcatatggcatgcatgtgctctgtatgtatataaaactcttgttttcttcttttctctaaatattctttccttatacattaggacctttgcagcataaattactatacttctatagacacacaaacacaaatacacacactaaattaata'
part12['need_reverse']=0 #if parts needs to be reversed:1 else:0
part12['Primer_up_U']='GV1' #user site up
part12['Primer_down_U']='U1' #user site down
part12['Primer_up']='cagcattttcaaaggtgt'
part12['Primer_up_Tm']=''
part12['Primer_down']=''
part12['Primer_down_Tm']=''

part13={}
part13['name']='pCFB1239CaCatA'
part13['seq']='acaatgtcccaagctttcaccgaatctgttaagacttctttgggtccaaatgctactccaagagctaaaaagttgattgcctctttggttcaacacgttcatgatttcgctagagaaaaccatttgactaccgaagattggttgtggggtgttgatttcattaacagaattggtcaaatgtccgactccagaagaaacgaaggtattttggtttgcgatatcatcggtttggaaaccttggttgatgctttgactaacgaatccgaacaatctaaccatacctcctctgctattttgggtcctttttacttgccagattctccagtttatccaaacggtggttctatcgttcaaaaggctattccaactgatgttaagtgcttcgttagaggtaaggttactgatactgaaggtaaaccattgggtggtgctcaattggaagtttggcaatgtaattctgctggtttctactctcaacaagctgatcatgatggtccagaattcaatttgagaggtactttcattaccgacgacgaaggtaattactccttcgaatgtttaagaccaacctcctatccaattccatacgatggtcctgctggtgatttgttgaaaatcatggatagacatccaaacagaccatcccatattcattggagagtttctcatccaggttaccatactttgatcacccaaatctatgatgctgaatgtccatacaccaacaacgattctgtttacgctgttaaggatgacatcatcgttcacttcgaaaaggttgataacaaggataaggatttggtcggtaaggtcgaatacaagttggattacgatatttccttggccaccgaatcctctattcaagaagctagagctgctgctaaagctagacaagatgctgaaatcaagttgtaa'
part13['need_reverse']=0 #if parts needs to be reversed:1 else:0
part13['Primer_up_U']='U1' #user site up
part13['Primer_down_U']='U2' #user site down
part13['Primer_up']=''
part13['Primer_up_Tm']=''
part13['Primer_down']=''
part13['Primer_down_Tm']=''

part14={}
part14['name']='CPS1t'
part14['seq']='gcgcaatgattgaatagtcaaagattttttttttttaattttttttttttaattttttttttttttcatagaactttttatttaaataaatcacgtctatatatgtatcagtataacgtaaaaaaaaaaacaccgtcagttaaacaaaacataaataaaaaaaaaaagaagtgtcaaatcaagtgtcaaatc'
part14['need_reverse']=0 #if parts needs to be reversed:1 else:0
part14['Primer_up_U']='U2' #user site up
part14['Primer_down_U']='GV2' #user site down
part14['Primer_up']='gcgcaatgattgaatagtcaaa'
part14['Primer_up_Tm']=''
part14['Primer_down']='gatttgacacttgatttgacacttcttt'
part14['Primer_down_Tm']=''

User4=[part6, part7,part8]
User5=[part9, part10,part11]
User6=[part12, part13,part14]

In [13]:
part15={}
part15['name']='pCfB1241_TDH3p:KpAroY.B'
part15['seq']='tcattcaatttcttgagcgaattgttcagcagttctcaaaccgttccatcttctagctttgtgataatccaaaccgaattggtccaaaactctggtaactatatggttggtgatgtcatcaacggtttctggatgattgtaataagctggcattggtggaaccatagctacacccattctagacaaggccaacatgttttccaaatggatagtagacaatggcatttctcttgggaccaagaccaactttctaccttctttcaaaacaacatcagcagctctaccaaccaaaccttcagcataaccagctctaataccagccaaggttttcatagagcatggaataacgatcataccatcagttctgaaagaaccagaagaaatagtagcagcttgatcagctggagaatgagaaaaatcagccaaagcagcaacttctctagcagtccatggagtttccaattcaatggtggtcttagcccatttagacataaccaaatgggtttcaacttctggcatatctctcaaagcttgcaacaaagcaacacccaatggagcacctgtagcaccagtcataccgatgatcaacttcattgttttacctgcactacctgcacttttgtttgtttatgtgtgtttattcgaaactaagttcttggtgttttaaaactaaaaaaaagactaactataaaagtagaatttaagaagtttaagaaatagatttacagaattacaatcaatacctaccgtctttatatacttattagtcaagtaggggaataatttcagggaactggtttcaaccttttttttcagctttttccaaatcagagagagcagaaggtaatagaaggtgtaagaaaatgagatagatacatgcgtgggtcaattgccttgtgtcatcatttactccaggcaggttgcatcactccattgaggttgtgcccgttttttgcctgtttgtgcccctgttctctgtagttgcgctaagagaatggacctatgaactgatggttggtgaagaaaacaatattttggtgctgggattctttttttttctggatgccagcttaaaaagcgggctccattatatttagtggatgccaggaataaactgttcacccagacacctacgatgttatatattctgtgtaacccgccccctattttgggcatgtacgggttacagcagaattaaaaggctaattttttgactaaataaagttaggaaaatcactactattaattatttacgtattctttgaaatggcagtattgataatgataaactcgaactgaaaaagcgtgttttttatagc'
#'caacggaatgcgtgcgattcattcaatttcttgagcgaattgttcagcagttctcaaaccgttccatcttctagctttgtgataatccaaaccgaattggtccaaaactctggtaactatatggttggtgatgtcatcaacggtttctggatgattgtaataagctggcattggtggaaccatagctacacccattctagacaaggccaacatgttttccaaatggatagtagacaatggcatttctcttgggaccaagaccaactttctaccttctttcaaaacaacatcagcagctctaccaaccaaaccttcagcataaccagctctaataccagccaaggttttcatagagcatggaataacgatcataccatcagttctgaaagaaccagaagaaatagtagcagcttgatcagctggagaatgagaaaaatcagccaaagcagcaacttctctagcagtccatggagtttccaattcaatggtggtcttagcccatttagacataaccaaatgggtttcaacttctggcatatctctcaaagcttgcaacaaagcaacacccaatggagcacctgtagcaccagtcataccgatgatcaacttcattgttttacctgcactacctgcacttttgtttgtttatgtgtgtttattcgaaactaagttcttggtgttttaaaactaaaaaaaagactaactataaaagtagaatttaagaagtttaagaaatagatttacagaattacaatcaatacctaccgtctttatatacttattagtcaagtaggggaataatttcagggaactggtttcaaccttttttttcagctttttccaaatcagagagagcagaaggtaatagaaggtgtaagaaaatgagatagatacatgcgtgggtcaattgccttgtgtcatcatttactccaggcaggttgcatcactccattgaggttgtgcccgttttttgcctgtttgtgcccctgttctctgtagttgcgctaagagaatggacctatgaactgatggttggtgaagaaaacaatattttggtgctgggattctttttttttctggatgccagcttaaaaagcgggctccattatatttagtggatgccaggaataaactgttcacccagacacctacgatgttatatattctgtgtaacccgccccctattttgggcatgtacgggttacagcagaattaaaaggctaattttttgactaaataaagttaggaaaatcactactattaattatttacgtattctttgaaatggcagtattgataatgataaactcgaactgaaaaagcgtgttttttatagc'
part15['need_reverse']=1#if parts needs to be reversed:1 else:0
part15['Primer_up_U']='GV1' #user site up
part15['Primer_down_U']='U2' #user site down
part15['Primer_up']=''
part15['Primer_up_Tm']=''
part15['Primer_down']=''
part15['Primer_down_Tm']=''


part16={}
part16['name']='VPS13t'
part16['seq']='gcgcgctgcggatatttctaaggcatggtcgtgcggagctacaataatacgattgaattatagctacatagtgtacaaaagcgggtatatactttcatatgtga'
part16['need_reverse']=1#if parts needs to be reversed:1 else:0
part16['Primer_up_U']='U2' #user site up
part16['Primer_down_U']='GV2' #user site down
part16['Primer_up']='' #we dont provide Curran's primer because their TM is very high for VPS13t
part16['Primer_up_Tm']=''
part16['Primer_down']='' #we dont provide Curran's primer because their TM is very high for VPS13t
part16['Primer_down_Tm']=''

User7=[part15, part16]


In [14]:
# this user assembly was redesigned to accomodate a gBlock for AroyB P146T


part1={}
part1['name']='pCfB1241_TDH3p'
part1['seq']='tttgtttgtttatgtgtgtttattcgaaactaagttcttggtgttttaaaactaaaaaaaagactaactataaaagtagaatttaagaagtttaagaaatagatttacagaattacaatcaatacctaccgtctttatatacttattagtcaagtaggggaataatttcagggaactggtttcaaccttttttttcagctttttccaaatcagagagagcagaaggtaatagaaggtgtaagaaaatgagatagatacatgcgtgggtcaattgccttgtgtcatcatttactccaggcaggttgcatcactccattgaggttgtgcccgttttttgcctgtttgtgcccctgttctctgtagttgcgctaagagaatggacctatgaactgatggttggtgaagaaaacaatattttggtgctgggattctttttttttctggatgccagcttaaaaagcgggctccattatatttagtggatgccaggaataaactgttcacccagacacctacgatgttatatattctgtgtaacccgccccctattttgggcatgtacgggttacagcagaattaaaaggctaattttttgactaaataaagttaggaaaatcactactattaattatttacgtattctttgaaatggcagtattgataatgataaactcgaactgaaaaagcgtgttttttatagc'
#'caacggaatgcgtgcgattcattcaatttcttgagcgaattgttcagcagttctcaaaccgttccatcttctagctttgtgataatccaaaccgaattggtccaaaactctggtaactatatggttggtgatgtcatcaacggtttctggatgattgtaataagctggcattggtggaaccatagctacacccattctagacaaggccaacatgttttccaaatggatagtagacaatggcatttctcttgggaccaagaccaactttctaccttctttcaaaacaacatcagcagctctaccaaccaaaccttcagcataaccagctctaataccagccaaggttttcatagagcatggaataacgatcataccatcagttctgaaagaaccagaagaaatagtagcagcttgatcagctggagaatgagaaaaatcagccaaagcagcaacttctctagcagtccatggagtttccaattcaatggtggtcttagcccatttagacataaccaaatgggtttcaacttctggcatatctctcaaagcttgcaacaaagcaacacccaatggagcacctgtagcaccagtcataccgatgatcaacttcattgttttacctgcactacctgcacttttgtttgtttatgtgtgtttattcgaaactaagttcttggtgttttaaaactaaaaaaaagactaactataaaagtagaatttaagaagtttaagaaatagatttacagaattacaatcaatacctaccgtctttatatacttattagtcaagtaggggaataatttcagggaactggtttcaaccttttttttcagctttttccaaatcagagagagcagaaggtaatagaaggtgtaagaaaatgagatagatacatgcgtgggtcaattgccttgtgtcatcatttactccaggcaggttgcatcactccattgaggttgtgcccgttttttgcctgtttgtgcccctgttctctgtagttgcgctaagagaatggacctatgaactgatggttggtgaagaaaacaatattttggtgctgggattctttttttttctggatgccagcttaaaaagcgggctccattatatttagtggatgccaggaataaactgttcacccagacacctacgatgttatatattctgtgtaacccgccccctattttgggcatgtacgggttacagcagaattaaaaggctaattttttgactaaataaagttaggaaaatcactactattaattatttacgtattctttgaaatggcagtattgataatgataaactcgaactgaaaaagcgtgttttttatagc'
part1['need_reverse']=1#if parts needs to be reversed:1 else:0
part1['Primer_up_U']='GV1' #user site up
part1['Primer_down_U']='U7' #user site down
part1['Primer_up']=''
part1['Primer_up_Tm']=''
part1['Primer_down']=''
part1['Primer_down_Tm']=''

part2={}
part2['name']='KpAroY.B-P146T.gBlock'
part2['seq']='aaacaatgaagttgatcatcggtatgactggtgctacaggtgctccattgggtgttgctttgttgcaagctttgagagatatgccagaagttgaaacccatttggttatgtctaaatgggctaagaccaccattgaattggaaactccatggactgctagagaagttgctgctttggctgatttttctcattctccagctgatcaagctgctactatttcttctggttctttcagaactgatggtatgatcgttattccatgctctatgaaaaccttggctggtattagagctggttatgctgaaggtttggttggtagagctgctgatgttgttttgaaagaaggtagaaagttggtcttggtcccaagagaaatgccattgtctactatccatttggaaaacatgttggccttgtctagaatgggtgtagctatggttAcaccaatgccagcttattacaatcatccagaaaccgttgatgacatcaccaaccatatagttaccagagttttggaccaattcggtttggattatcacaaagctagaagatggaacggtttgagaactgctgaacaattcgctcaagaaattgaatga'
#'caacggaatgcgtgcgattcattcaatttcttgagcgaattgttcagcagttctcaaaccgttccatcttctagctttgtgataatccaaaccgaattggtccaaaactctggtaactatatggttggtgatgtcatcaacggtttctggatgattgtaataagctggcattggtggaaccatagctacacccattctagacaaggccaacatgttttccaaatggatagtagacaatggcatttctcttgggaccaagaccaactttctaccttctttcaaaacaacatcagcagctctaccaaccaaaccttcagcataaccagctctaataccagccaaggttttcatagagcatggaataacgatcataccatcagttctgaaagaaccagaagaaatagtagcagcttgatcagctggagaatgagaaaaatcagccaaagcagcaacttctctagcagtccatggagtttccaattcaatggtggtcttagcccatttagacataaccaaatgggtttcaacttctggcatatctctcaaagcttgcaacaaagcaacacccaatggagcacctgtagcaccagtcataccgatgatcaacttcattgttttacctgcactacctgcacttttgtttgtttatgtgtgtttattcgaaactaagttcttggtgttttaaaactaaaaaaaagactaactataaaagtagaatttaagaagtttaagaaatagatttacagaattacaatcaatacctaccgtctttatatacttattagtcaagtaggggaataatttcagggaactggtttcaaccttttttttcagctttttccaaatcagagagagcagaaggtaatagaaggtgtaagaaaatgagatagatacatgcgtgggtcaattgccttgtgtcatcatttactccaggcaggttgcatcactccattgaggttgtgcccgttttttgcctgtttgtgcccctgttctctgtagttgcgctaagagaatggacctatgaactgatggttggtgaagaaaacaatattttggtgctgggattctttttttttctggatgccagcttaaaaagcgggctccattatatttagtggatgccaggaataaactgttcacccagacacctacgatgttatatattctgtgtaacccgccccctattttgggcatgtacgggttacagcagaattaaaaggctaattttttgactaaataaagttaggaaaatcactactattaattatttacgtattctttgaaatggcagtattgataatgataaactcgaactgaaaaagcgtgttttttatagc'
part2['need_reverse']=0 #if parts needs to be reversed:1 else:0
part2['Primer_up_U']='U7' #user site up
part2['Primer_down_U']='U2' #user site down
part2['Primer_up']=''
part2['Primer_up_Tm']=''
part2['Primer_down']=''
part2['Primer_down_Tm']=''

part3={}
part3['name']='VPS13t'
part3['seq']='gcgcgctgcggatatttctaaggcatggtcgtgcggagctacaataatacgattgaattatagctacatagtgtacaaaagcgggtatatactttcatatgtga'
part3['need_reverse']=1#if parts needs to be reversed:1 else:0
part3['Primer_up_U']='U2' #user site up
part3['Primer_down_U']='GV2' #user site down
part3['Primer_up']='' #we dont provide Curran's primer because their TM is very high for VPS13t
part3['Primer_up_Tm']=''
part3['Primer_down']='' #we dont provide Curran's primer because their TM is very high for VPS13t
part3['Primer_down_Tm']=''

pCCM011=[part1, part2, part3]

In [15]:

part17={}
part17['name']='pCFB1241_Tef1p_KpAroyC.iso'
part17['seq']='gcacacaccatagcttcaaaatgtttctactccttttttactcttccagattttctcggactccgcgcatcgccgtaccacttcaaaacacccaagcacagcatactaaatttcccctctttcttcctctagggtgtcgttaattacccgtactaaaggtttggaaaagaaaaaagagaccgcctcgtttctttttcttcgtcgaaaaaggcaataaaaatttttatcacgtttctttttcttgaaaatttttttttttgatttttttctctttcgatgacctcccattgatatttaagttaataaacggtcttcaatttctcaagtttcagtttcatttttcttgttctattacaactttttttacttcttgctcattagaaagaaagcatagcaatctaatctaagttttaattacaaatctgtcatatctgtcataaaacaatgaccgccccaatccaagatttgagagatgctattgctttgttacaacaacacgacaatcaatacttggaaaccgatcatccagttgatccaaatgctgaattggctggtgtttacagacatattggtgctggtggtactgtaaaaagaccaactagaattggtccagccatgatgttcaacaacattaagggttatccacactccagaatcttggttggtatgcatgcttctagacaaagagcagctttgttgttgggttgtgaagcttctcaattggctttggaagttggtaaagctgttaagaaaccagttgctccagttgttgttccagcttcttctgctccatgtcaagaacaaattttcttggctgatgatccagacttcgatttgagaactttgttgccagctcataccaacactccaattgatgctggtccatttttttgtttgggtttggctttagcttctgatcctgttgatgcttctttgaccgatgttaccattcatagattgtgcgttcaaggtagagatgaattgtctatgtttttggctgccggtagacatatcgaagtttttagacaaaaagctgaagctgctggtaagccattgccaattactattaacatgggtttagatccagccatctacattggtgcttgttttgaagctccaactactccatttggttacaacgaattgggtgttgctggtgctttgagacaaagaccagttgaattggttcaaggtgtttctgttccagaaaaggctattgctagagccgaaatagttatcgaaggtgaattattgccaggtgtcagagttagagaagatcaacatacaaattccggtcatgctatgccagaatttccaggttattgtggtggtgctaatccatctttgccagttattaaggttaaggccgttaccatgagaaacaacgctattttacaaactttggtcggtccaggtgaagaacatacaactttggctggtttgccaaccgaagcttctatttggaatgctgttgaagctgcaattccaggtttcttgcaaaatgtttatgctcatacagctggtggtggtaagttcttgggtatattgcaagtcaagaaaagacaaccagctgacgaaggtagacaaggtcaagctgctttattagctttggctacttactccgaattgaagaatatcatcttggtcgatgaagatgttgatatcttcgattccgatgatattttgtgggctatgactactagaatgcaaggtgatgtttccattactaccattccaggtattagaggtcaccaattagatccatctcaaaccccagaatactccccatcaattagaggtaatggtatctcctgtaagaccattttcgattgcactgttccatgggctttgaagtctcattttgaaagagcaccatttgctgacgttgatcctagaccttttgctccagaatatttcgctagattggaaaagaatcaaggttccgctaagtaa'
part17['need_reverse']=0 #if parts needs to be reversed:1 else:0
part17['Primer_up_U']='GV1' #user site up
part17['Primer_down_U']='U2' #user site down
part17['Primer_up']=''
part17['Primer_up_Tm']=''
part17['Primer_down']=''
part17['Primer_down_Tm']=''



part18={}
part18['name']='IDP1t'
part18['seq']='tcgaatttacgtagcccaatctaccacttttttttttcattttttaaagtgttatacttagttatgctctaggataatgaactacttttttttttttttttttactgttatcataaatatatataccttattgttgtttgcaaccgtcggttaattccttatcaaggttccccaagttcggatcattaccatc'
part18['need_reverse']=0 #if parts needs to be reversed:1 else:0
part18['Primer_up_U']='U2' #user site up
part18['Primer_down_U']='GV2' #user site down
part18['Primer_up']='tcgaatttacgtagcccaatcta'
part18['Primer_up_Tm']=''
part18['Primer_down']='gatggtaatgatccgaacttgg'
part18['Primer_down_Tm']=''
User8=[part17, part18]

In [16]:
part19={}
part19['name']='CCW12p'
part19['seq']='tattgatatagtgtttaagcgaatgacagaagattaatttcttggtatgttaggaaagaataaaggagaataagaataattagaacaatgtaggatggaaagaaagattatcaagcatgccgactttatatacttgaacggaggcaaaggatgcaaaattttctcacatttctttctgccgttatgttggaagtaagactcccattatcgcaatactgcaacacgaatatgcaaaatttgctgagttatcgcagatagttgttgcaaagatagcggcgtaggtggccgcgaaatggggaattccaaaacaaacggtttttttactcctgagaaatacttgtacgggataatccagggcctaccacccacgcttcgaggattggcttttatttttttttttttggtggcgttttatttctttcccgctttctgggacttgtgcggagttttgagaggggcgcgcggcaaaggattcccaaaacggaaatcagacgccaatagccagcactcaaagcagttctggacccattccgattttcccatttggttcttgcgcgtgctgattccgacacgcgcgtctataaatagcatgaagtatccgcacaccgcagcgttagtgaggtgagggtggcagcaagctaattcccgcatctggaatctgaactgccccttttggactaaccgtgtggttcatgggtgggcgaagtgcgcaacctcgaaggttttcttttgcgtgtcggattttacatccggcggtagcgcatgatgccatggctggctccagatacatcctcagggcaccagcatctataattagattggcgcaacatggctggctgcactgctgtcttcacttctttctttttccggcaatgaatgatgtatgttttgtggcaaaagggtccgcattgtacctgtttacagttgagattatcgtttttggtagcccttcattacggcataacgtattaagtttcttt'
part19['need_reverse']=1 #if parts needs to be reversed:1 else:0
part19['Primer_up_U']='GV1' #user site up
part19['Primer_down_U']='U1' #user site down
part19['Primer_up']=''
part19['Primer_up_Tm']=''
part19['Primer_down']='aaagaaacttaatacgttatgcc'
part19['Primer_down_Tm']=''

part20={}
part20['name']='pCfB1237_KpAroY.D'
part20['seq']='acaatgatctgtccaagatgcgccgacgaaaaaattgaagttatggctacttctccagttaagggtgtttggactgtttatcaatgtcaacactgcttgtacacttggagagatactgaaccattgagaagaacctctagagaacattaccctgaagctttcagaatgacccaaaaggatattgatgaagctccacaagttcctcatgttccaccattattgccagaagataagagataa'
part20['need_reverse']=0 #if parts needs to be reversed:1 else:0
part20['Primer_up_U']='U1' #user site up
part20['Primer_down_U']='U2' #user site down
part20['Primer_up']=''
part20['Primer_up_Tm']=''
part20['Primer_down']=''
part20['Primer_down_Tm']=''

part21={}
part21['name']='PRM9t'
part21['seq']='acagaagacgggagacactagcacacaactttaccaggcaaggtatttgacgctagcatgtgtccaattcagtgtcatttatgattttttgtagtaggatataaatatatacagcgctccaaatagtgcggttgccccaaaaacaccacggaacctcatctgttctcgtactttgttgtgacaaagtagctcactgccttattatcacattttcattatgcaacgcttcggaaaatacgatgttgaaaat'
part21['need_reverse']=0 #if parts needs to be reversed:1 else:0
part21['Primer_up_U']='U2' #user site up
part21['Primer_down_U']='GV2' #user site down
part21['Primer_up']='' #we dont provide Curran's primer because their TM is very high for PRM9t
part21['Primer_up_Tm']=''
part21['Primer_down']='' #we dont provide Curran's primer because their TM is very high for PRM9t
part21['Primer_down_Tm']=''
User9=[part19, part20, part21]

In [17]:
CCM004={}
CCM004['name']='pCCM004(pFBA1:PaAroZ:tHis5)'
CCM004['seq']='ACTGGTAGAGAGCGACTTTgtatgccccaattgcgaaacccgcgatatccttctcgattctttagtacccgaccaggacaaggaaaaggaggtcgaaacgtttttgaagaaacaagaggaactacacggaagctctaaagatggcaaccagccagaaactaagaaaatgaagttgatggatccaactggcaccgctggcttgaacaacaataccagccttccaacttctgtaaataacggcggtacgccagtgccaccagtaccgttacctttcggtatacctcctttccccatgtttccaatgcccttcatgcctccaacggctactatcacaaatcctcatcaagctgacgcaagccctaagaaatgaataacaatactgacagtactaaataattgcctacttggcttcacatacgttgcatacgtcgatatagataataatgataatgacagcaggattatcgtaatacgtaatagttgaaaatctcaaaaatgtgtgggtcattacgtaaataatgataggaatgggattcttctatttttcctttttccattctagcagccgtcgggaaaacgtggcatcctctctttcgggctcaattggagtcacgctgccgtgagcatcctctctttccatatctaacaactgagcacgtaaccaatggaaaagcatgagcttagcgttgctccaaaaaagtattggatggttaataccatttgtctgttctcttctgactttgactcctcaaaaaaaaaaaatctacaatcaacagatcgcttcaattacgccctcacaaaaacttttttccttcttcttcgcccacgttaaattttatccctcatgttgtctaacggatttctgcacttgatttattataaaaagacaaagacataatacttctctatcaatttcagttattgttcttccttgcgttattcttctgttcttctttttcttttgtcatatataacCATAACCAAGTAATACATATTCAAAAGCCCTCTtACAATGCCATCCAAGTTGGccattacctctatgtctttgggtagatgttatgccggtcattctttcactactaagttggatatggctagaaagtacggttaccaaggtttggaattattccatgaagatttggctgatgtcgcctatagattgtctggtgaaactccatctccatgtggtccatcaccagctgctcaattgtctgctgctagacaaattttgagaatgtgccaagtcagaaacatcgaaatcgtttgcttgcaaccattctctcaatacgatggtttgttggatagagaagaacacgaaagaagattggaacaattggaattctggatcgaattggcccatgaattggataccgatattattcaaattccagccaacttcttgccagccgaagaagttactgaagatatctctttgattgtctccgacttgcaagaagtagctgatatgggtttacaagctaacccaccaattagattcgtttacgaagctttgtgttggtccactagagttgatacttgggaaagatcttgggaagttgttcaaagagttaacagaccaaacttcggtgtttgtttggacacttttaacattgccggtagagtttatgctgatccaactgttgcttctggtagaactccaaatgctgaagaagctatcagaaagtccattgccagattggttgaaagagttgacgtttccaaggttttctacgttcaagttgttgatgccgaaaagttgaagaaaccattggttccaggtcacagattctatgatccagaacaaccagctagaatgtcttggtctagaaactgcagattattctacggtgaaaaggatagaggtgcttacttgccagtaaaagaaattgcttgggcttttttcaacggtttgggttttgaaggttgggtttccttagaattattcaacagaagaatgtccgataccggttttggtgttccagaagaattagctagaagaggtgctgtttcttgggctaaattggttagagatatgaagatcaccgttgactctccaactcaacaacaagctacacaacaacctatcagaatgttgtCTTTGTCAGCTGCTTTGTGAACACAGGCtATAGATTAATTTAAACAGTATATGTACAGTTTtatatatatatatatatatatatacatatataaagaaacctgtgcgttttttgtattttcaaatacatttagttttggcgccacttcTATAAAAGGTCTCATGATATTGTTAC'
CCM004['need_reverse']=0 #if parts needs to be reversed:1 else:0
CCM004['Primer_up_U']='GV1' #user site up
CCM004['Primer_down_U']='U1'#user site down
CCM004['Primer_up']=''
CCM004['Primer_up_Tm']=''
CCM004['Primer_down']=''
CCM004['Primer_down_Tm']=''

CCM007={}
CCM007['name']='pCCM007(pTDH3:KpAroyB:tVPS13)'
CCM007['seq']='ATAAAAAACACGCTTTTTCAgttcgagtttatcattatcaatactgccatttcaaagaatacgtaaataattaatagtagtgattttcctaactttatttagtcaaaaaattagccttttaattctgctgtaacccgtacatgcccaaaatagggggcgggttacacagaatatataacatcgtaggtgtctgggtgaacagtttattcctggcatccactaaatataatggagcccgctttttaagctggcatccagaaaaaaaaagaatcccagcaccaaaatattgttttcttcaccaaccatcagttcataggtccattctcttagcgcaactacagagaacaggggcacaaacaggcaaaaaacgggcacaacctcaatggagtgatgcaacctgcctggagtaaatgatgacacaaggcaattgacccacgcatgtatctatctcattttcttacaccttctattaccttctgctctctctgatttggaaaaagctgaaaaaaaaggttgaaaccagttccctgaaattattcccctacttgactaataagtatataaagacggtaggtattgattgtaattctgtaaatctatttcttaaacttcttaaattctacttttatagttagtcttttttttagttttaaaacaccaagaacttagtttcgaataaacacacataaacaaacaaaagtgcaggtagtgcaggtaaaacaatgaagttgatcatcggtatgactggtgctacaggtgctccattgggtgttgctttgttgcaagctttgagagatatgccagaagttgaaacccatttggttatgtctaaatgggctaagaccaccattgaattggaaactccatggactgctagagaagttgctgctttggctgatttttctcattctccagctgatcaagctgctactatttcttctggttctttcagaactgatggtatgatcgttattccatgctctatgaaaaccttggctggtattagagctggttatgctgaaggtttggttggtagagctgctgatgttgttttgaaagaaggtagaaagttggtcttggtcccaagagaaatgccattgtctactatccatttggaaaacatgttggccttgtctagaatgggtgtagctatggttccaccaatgccagcttattacaatcatccagaaaccgttgatgacatcaccaaccatatagttaccagagttttggaccaattcggtttggattatcacaaagctagaagatggaacggtttgagaactgctgaacaattcgctcaagaaattgaatgaatCGCACGCATTCCGTTGACACAGGCtTCACATATGAAAGTATATACCCGcttttgtacactatgtagctataattcaatcgtattattgtagctccgcacgaccatgccttagaaataTCCGCAGCGCGC'
CCM007['need_reverse']=0 #if parts needs to be reversed:1 else:0
CCM007['Primer_up_U']='U1' #user site up
CCM007['Primer_down_U']='U2'#user site down
CCM007['Primer_up']=''
CCM007['Primer_up_Tm']=''
CCM007['Primer_down']=''
CCM007['Primer_down_Tm']=''

CCM008={}
CCM008['name']='pCCM008(pTEF1:Aroy.Ciso:tIDP1)'
CCM008['seq']='GCACACACCATAGCTTCAAaatgtttctactccttttttactcttccagattttctcggactccgcgcatcgccgtaccacttcaaaacacccaagcacagcatactaaatttcccctctttcttcctctagggtgtcgttaattacccgtactaaaggtttggaaaagaaaaaagagaccgcctcgtttctttttcttcgtcgaaaaaggcaataaaaatttttatcacgtttctttttcttgaaaatttttttttttgatttttttctctttcgatgacctcccattgatatttaagttaataaacggtcttcaatttctcaagtttcagtttcatttttcttgttctattacaactttttttacttcttgctcattagaaagaaagcatagcaatctaatctaagttttaattacaatctgtcataaaacaatgaccgccccaatccaagatttgagagatgctattgctttgttacaacaacacgacaatcaatacttggaaaccgatcatccagttgatccaaatgctgaattggctggtgtttacagacatattggtgctggtggtactgtaaaaagaccaactagaattggtccagccatgatgttcaacaacattaagggttatccacactccagaatcttggttggtatgcatgcttctagacaaagagcagctttgttgttgggttgtgaagcttctcaattggctttggaagttggtaaagctgttaagaaaccagttgctccagttgttgttccagcttcttctgctccatgtcaagaacaaattttcttggctgatgatccagacttcgatttgagaactttgttgccagctcataccaacactccaattgatgctggtccatttttttgtttgggtttggctttagcttctgatcctgttgatgcttctttgaccgatgttaccattcatagattgtgcgttcaaggtagagatgaattgtctatgtttttggctgccggtagacatatcgaagtttttagacaaaaagctgaagctgctggtaagccattgccaattactattaacatgggtttagatccagccatctacattggtgcttgttttgaagctccaactactccatttggttacaacgaattgggtgttgctggtgctttgagacaaagaccagttgaattggttcaaggtgtttctgttccagaaaaggctattgctagagccgaaatagttatcgaaggtgaattattgccaggtgtcagagttagagaagatcaacatacaaattccggtcatgctatgccagaatttccaggttattgtggtggtgctaatccatctttgccagttattaaggttaaggccgttaccatgagaaacaacgctattttacaaactttggtcggtccaggtgaagaacatacaactttggctggtttgccaaccgaagcttctatttggaatgctgttgaagctgcaattccaggtttcttgcaaaatgtttatgctcatacagctggtggtggtaagttcttgggtatattgcaagtcaagaaaagacaaccagctgacgaaggtagacaaggtcaagctgctttattagctttggctacttactccgaattgaagaatatcatcttggtcgatgaagatgttgatatcttcgattccgatgatattttgtgggctatgactactagaatgcaaggtgatgtttccattactaccattccaggtattagaggtcaccaattagatccatctcaaaccccagaatactccccatcaattagaggtaatggtatctcctgtaagaccattttcgattgcactgttccatgggctttgaagtctcattttgaaagagcaccatttgctgacgttgatcctagaccttttgctccagaatatttcgctagattggaaaagAATCAAGGTTCCGCTAAGTAAACACAGGCtTCGAATTTACGTAGCCCAATCTAccacttttttttttcattttttaaagtgttatacttagttatgctctaggataatgaactacttttttttttttttttttactgttatcataaatatatataccttattgttgtttgcaaccgtcggttaattccttatcaaggttccCCAAGTTCGGATCATTACCATC'
CCM008['need_reverse']=0 #if parts needs to be reversed:1 else:0
CCM008['Primer_up_U']='U2' #user site up
CCM008['Primer_down_U']='GV2'#user site down
CCM008['Primer_up']=''
CCM008['Primer_up_Tm']=''
CCM008['Primer_down']=''
CCM008['Primer_down_Tm']=''

pCCM012_13_14_15=[CCM004,CCM007,CCM008]

In [18]:
CCM009={}
CCM009['name']='pCCM009(pCCW12:AroYD:tPRM9)'
CCM009['seq']='AAAGAAACTTAATACGTTATGCCgtaatgaagggctaccaaaaacgataatctcaactgtaaacaggtacaatgcggacccttttgccacaaaacatacatcattcattgccggaaaaagaaagaagtgaagacagcagtgcagccagccatgttgcgccaatctaattatagatgctggtgccctgaggatgtatctggagccagccatggcatcatgcgctaccgccggatgtaaaatccgacacgcaaaagaaaaccttcgaggttgcgcacttcgcccacccatgaaccacacggttagtccaaaaggggcagttcagattccagatgcgggaattagcttgctgccaccctcacctcactaacgctgcggtgtgcggatacttcatgctatttatagacgcgcgtgtcggaatcagcacgcgcaagaaccaaatgggaaaatcggaatgggtccagaactgctttgagtgctggctattggcgtctgatttccgttttgggaatcctttgccgcgcgcccctctcaaaactccgcacaagtcccagaaagcgggaaagaaataaaacgccaccaaaaaaaaaaaaataaaagccaatcctcgaagcgtgggtggtaggccctggattatcccgtacaagtatttctcaggagtaaaaaaaccgtttgttttggaattccccatttcgcggccacctacgccgctatctttgcaacaactatctgcgataactcagcaaattttgcatattcgtgttgcagtattgcgataatgggagtcttacttccaacataacggcagaaagaaatgtgagaaaattttgcatcctttgcctccgttcaagtatataaagtcggcatgcttgataatctttctttccatcctacattgttctaattattcttattctcctttattctttcctaacataccaagaaattaatcttctgTCATTCGCTTAAACACTATATCAATAAGCCCTCTtACAATGATCTGTCCAAGATGCgccgacgaaaaaattgaagttatggctacttctccagttaagggtgtttggactgtttatcaatgtcaacactgcttgtacacttggagagatactgaaccattgagaagaacctctagagaacattaccctgaagctttcagaatgacccaaaaggatattgatgaagctccacaagttcctcatgttccacCATTATTGCCAGAAGATAAGAGATAAACACAGGCtACAGAAGACGGGAGACACtagcacacaactttaccaggcaaggtatttgacgctagcatgtgtccaattcagtgtcatttatgattttttgtagtaggatataaatatatacagcgctccaaatagtgcggttgccccaaaaacaccacggaacctcatctgttctcgtactttgttgtgacaaagtagctcactgccttattatcacattttcattatgcaacgctTCGGAAAATACGATGTTGAAAAT'
CCM009['need_reverse']=0 #if parts needs to be reversed:1 else:0
CCM009['Primer_up_U']='GV1' #user site up
CCM009['Primer_down_U']='U1'#user site down
CCM009['Primer_up']=''
CCM009['Primer_up_Tm']=''
CCM009['Primer_down']=''
CCM009['Primer_down_Tm']=''

CCM005={}
CCM005['name']='pCCM005(pTPI1:NrsR:LSC2t)'
CCM005['seq']='tAAGGATGAGCCAAGAATAAGGgaacaagattttggtaatttccaaaaaatcaatagcatgcaggacgttatgaagaagagatctacgtatggtcatttcttcttcagattccctcatggagaaagtgcggcagatgtatatgacagagtcgccagtttccaagagactttattcaggcacttccatgataggcaagagagaagacccagagatgttgttgtcctagttacacatggtatttattccagagtattcctgatgaaatggtttagatggacatacgaagagtttgaatcgtttaccaatgttcctaacgggagcgtaatggtgatggaactggacgaatccatcaatagatacgtcctgaggaccgtgctacccaaatggactgattgtgagggagacctaactacatagtgtttaaagattacggatatttaacttacttagaataatgccatttttttgagttataataatcctacgttagtgtgagcgggatttaaactgtgaggaccttaatacattcagacacttctgcggtatcaccctacttattcccttcgagattatatctaggaacccatcaggttggtggaagattacccgttctaagacttttcagcttcctctattgatgttacacctggacaccccttttctggcatccagtttttaatcttcagtggcatgtgagattctccgaaattaattaaagcaatcacacaattctctcggataccacctcggttgaaactgacaggtggtttgttacgcatgctaatgcaaaggagcctatatacctttggctcggctgctgtaacagggaatataaagggcagcataatttaggagtttagtgaacttgcaacatttactattttcccttcttacgtaaatatttttctttttaattctaaatcaatctttttcaattttttgtttgtattcttttcttgcttaaatctataACTACAAAAAACACATACATAAACTAAAAAGCCCTCTtACCATGGGTACCACTCTtgacgacacggcttaccggtaccgcaccagtgtcccgggggacgccgaggccatcgaggcactggatgggtccttcaccaccgacaccgttttccgcgtcaccgccaccggggacggcttcaccctgcgggaggtgccggtggacccgcccctgaccaaggtgttccccgacgacgaatcggacgacgaatcggacgacggggaggacggcgacccggactcccggacgttcgtcgcgtacggggacgacggcgacctggcgggcttcgtggtcgtctcgtactccggctggaaccgccggctgaccgtcgaggacatcgaggtcgccccggagcaccgggggcacggggtcgggcgcgcgttgatggggctcgcgacggagttcgcccgcgagcggggcgccgggcacctctggctggaggtcaccaacgtcaacgcaccggcgatccacgcgtaccggcggatggggttcaccctctgcggcctggacaccgccctgtacgacggcaccgcctcggacggcgagcaggcgctctacatgagcaTGCCCTGCCCCTAAACACAGGCtGCTTCTCGAGAAAAACAAAAGAGTTAataataaagtatatatgcttttttactattaataaagtttcttattttccctacatgtatacatacatataaatatactcttttGGAAAAAATTTCTTTTTTTTTTTTTTAATTTT'
CCM005['need_reverse']=0 #if parts needs to be reversed:1 else:0
CCM005['Primer_up_U']='U1' #user site up
CCM005['Primer_down_U']='U2'#user site down
CCM005['Primer_up']=''
CCM005['Primer_up_Tm']=''
CCM005['Primer_down']=''
CCM005['Primer_down_Tm']=''

CCM006={}
CCM006['name']='pCCM006(pCyc1:CaCatA:CPS1t)'
CCM006['seq']='tCAGCATTTTCAAAGGTGTgttcttcgtcagacatgttttagtgtgtgaatgaaataggtgtatgttttctttttgctagacaataattaggaacaaggtaagggaactaaagtgtagaataagattaaaaaagaagaacaagttgaaaaggcaagttgaaatttcaagaaaaaagtcaattgaagtacagtaaattgacctgaatatatctgagttccgacaacaatgagtttaccaaagagaacaatggaataggaaactttgaacgaagaaaggaaagcaggaaaggaaaaaatttttaggctcgagaacaatagggcgaaaaaacaggcaacgaacgaacaatggaaaaacgaaaaaaaaaaaaaaaaacacagaaaagaatgcagaaagatgtcaactgaaaaaaaaaaaggtgaacacaggaaaaaaaataaaaaaaaaaaaaaaaaaaggaggacgaaacaaaaaagtgaaaaaaaatgaaaatttttttggaaaaccaagaaatgaattatatttccgtgtgagacgacatcgtcgaatatgattcagggtaacagtattgatgtaatcaatttcctacctgaatctaaaattcccgggagcaagatcaagatgttttcaccgatctttccggtctctttggccggggtttacggacgatggcagaagaccaaagcgccagttcatttggcgagcgttggttggtggatcaagcccacgcgtaggcaatcctcgagcagatccgccaggcgtgtatatatagcgtggatggccaggcaactttagtgctgacacatacaggcatatatatatgtgtgcgacgacacatgatcatatggcatgcatgtgctctgtatgtatataaaactcttgttttcttcttttctctaaatattctttccttatacattaggacctttgcagcataaattactatacttctatagacacacaAACACAAATACACACACTAAATTAATAAGCCCTCTtACAATGTCCCAAGCTTTCAccgaatctgttaagacttctttgggtccaaatgctactccaagagctaaaaagttgattgcctctttggttcaacacgttcatgatttcgctagagaaaaccatttgactaccgaagattggttgtggggtgttgatttcattaacagaattggtcaaatgtccgactccagaagaaacgaaggtattttggtttgcgatatcatcggtttggaaaccttggttgatgctttgactaacgaatccgaacaatctaaccatacctcctctgctattttgggtcctttttacttgccagattctccagtttatccaaacggtggttctatcgttcaaaaggctattccaactgatgttaagtgcttcgttagaggtaaggttactgatactgaaggtaaaccattgggtggtgctcaattggaagtttggcaatgtaattctgctggtttctactctcaacaagctgatcatgatggtccagaattcaatttgagaggtactttcattaccgacgacgaaggtaattactccttcgaatgtttaagaccaacctcctatccaattccatacgatggtcctgctggtgatttgttgaaaatcatggatagacatccaaacagaccatcccatattcattggagagtttctcatccaggttaccatactttgatcacccaaatctatgatgctgaatgtccatacaccaacaacgattctgtttacgctgttaaggatgacatcatcgttcacttcgaaaaggttgataacaaggataaggatttggtcggtaaggtcgaatacaagttggattacgatatttccttggccaccgaatcctctattcaagaagctagagctgctgctaaagctagaCAAGATGCTGAAATCAAGTTGTAAACACAGGCtGCGCAATGATTGAATAGTCAAAgattttttttttttaattttttttttttaattttttttttttttcatagaactttttatttaaataaatcacgtctatatatgtatcagtataacgtaaaaaaaaaaacaccgtcagttaaacaaaacataaataaaaaaaaAAAGAAGTGTCAAATCAAGTGTCAAATC'
CCM006['need_reverse']=0 #if parts needs to be reversed:1 else:0
CCM006['Primer_up_U']='U2' #user site up
CCM006['Primer_down_U']='U3'#user site down
CCM006['Primer_up']=''
CCM006['Primer_up_Tm']=''
CCM006['Primer_down']=''
CCM006['Primer_down_Tm']=''

CCM003={}
CCM003['name']='pCCM003(pTDH2:Tkl1:TDH3t)'
CCM003['seq']='ATCTAGATCAGAGGGTGGTaaatgaagtgtaatagtattcatttttcttataaatcatcccttccgtgatttatacaaaagaagaggagaatatgctgaatacttggtatattactctacattatactcttatcttgacgggtattctgagcatcttactcagtttcaagatcttttaatgtccaaaaacatttgagccgatctaaatacttctgtgttttcattaatttataaattgtactcttttaagacatggaaagtaccaacatcggttgaaacagtttttcatttacttatggtttattggtttttccagtgaatgattatttgtcgttaccctttcgtaaaagttcaaacacgtttttaagtattgtttagttgctctttcgacatatatgattatccctgcgcggctaaagttaaggatgcaaaaaacataagacaactgaagttaatttacgtcaattaagttttccagggtaatgatgttttgggcttccactaattcaataagtatgtcatgaaatacgttgtgaagagcatccagaaataatgaaaagaaacaacgaaactgggtcggcctgttgtttcttttctttaccacgtgatctgcggcatttacaggaagtcgcgcgttttgcgcagttgttgcaacgcagctacggctaacaaagcctagtggaactcgactgatgtgttagggcctaaaactggtggtgacagctgaagtgaactattcaatccaatcatgtcatggctgtcacaaagaccttgcggaccgcacgtacgaacacatacgtatgctaatatgtgttttgatagtacccagtgatcgcagacctgcaatttttttgtaggtttggaagaatatataaaggttgcactcattcaagatagtttttttcttgtgtgtctattcattttattattgtttgtttaaatgttaaaaaaaccaagaacttagtttcaaaTTAAATTCATCACACAAACAAACAAAACAAAAGCCCTCTtACAATGACTCAATTCACTGACATTGATaagctagccgtctccaccataagaattttggctgtggacaccgtatccaaggccaactcaggtcacccaggtgctccattgggtatggcaccagctgcacacgttctatggagtcaaatgcgcatgaacccaaccaacccagactggatcaacagagatagatttgtcttgtctaacggtcacgcggtcgctttgttgtattctatgctacatttgactggttacgatctgtctattgaagacttgaaacagttcagacagttgggttccagaacaccaggtcatcctgaatttgagttgccaggtgttgaagttactaccggtccattaggtcaaggtatctccaacgctgttggtatggccatggctcaagctaacctggctgccacttacaacaagccgggctttaccttgtctgacaactacacctatgttttcttgggtgacggttgtttgcaagaaggtatttcttcagaagcttcctccttggctggtcatttgaaattgggtaacttgattgccatctacgatgacaacaagatcactatcgatggtgctaccagtatctcattcgatgaagatgttgctaagagatacgaagcctacggttgggaagttttgtacgtagaaaatggtaacgaagatctagccggtattgccaaggctattgctcaagctaagttatccaaggacaaaccaactttgatcaaaatgaccacaaccattggttacggttccttgcatgccggctctcactctgtgcacggtgccccattgaaagcagatgatgttaaacaactaaagagcaaattcggtttcaacccagacaagtcctttgttgttccacaagaagtttacgaccactaccaaaagacaattttaaagccaggtgtcgaagccaacaacaagtggaacaagttgttcagcgaataccaaaagaaattcccagaattaggtgctgaattggctagaagattgagcggccaactacccgcaaattgggaatctaagttgccaacttacaccgccaaggactctgccgtggccactagaaaattatcagaaactgttcttgaggatgtttacaatcaattgccagagttgattggtggttctgccgatttaacaccttctaacttgaccagatggaaggaagcccttgacttccaacctccttcttccggttcaggtaactactctggtagatacattaggtacggtattagagaacacgctatgggtgccataatgaacggtatttcagctttcggtgccaactacaaaccatacggtggtactttcttgaacttcgtttcttatgctgctggtgccgttagattgtccgctttgtctggccacccagttatttgggttgctacacatgactctatcggtgtcggtgaagatggtccaacacatcaacctattgaaactttagcacacttcagatccctaccaaacattcaagtttggagaccagctgatggtaacgaagtttctgccgcctacaagaactctttagaatccaagcatactccaagtatcattgctttgtccagacaaaacttgccacaattggaaggtagctctattgaaagcgcttctaagggtggttacgtactacaagatgttgctaacccagatattattttagtggctactggttccgaagtgtctttgagtgttgaagctgctaagactttggccgcaaagaacatcaaggctcgtgttgtttctctaccagatttcttcacttttgacaaacaacccctagaatacagactatcagtcttaccagacaacgttccaatcatgtctgttgaagttttggctaccacatgttggggcaaatacgctcatcaatccttcggtattgacagatttggtgcctccggtaaggcaccagaagtcttcaagttcttcggtttcaccccagaaggtgttgctgaaagagctcaaaagaccattgcattctataagggtgacaagctaatttcTCCTTTGAAAAAAGCTTTCTGAACACAGGCtGTGAATTTACTTTAAATCTTGCATTTAAATAAATTTTCTTTTTATAGCTTTATGACTTAGTTTCAATTTATATACTATTTTAATGACATTTTCGATTCATTGATTGAAAGCTTTGTGTTTTTTCTTGATGCGCTATTGCATTGTTCTTGTCTTTTTCGCCACATGTAATATCTGTAGTAGATACCTGATACATTGTGGATGCTGAGTGAAATTTTAGTTAATAATGGAGGCGCTCTTAATAATTTTGGGGATATTGGCTTTTTTTTTTAAAGTTTACAAATGAATTTTTTCCGCCAGGATAACGATTCTGAAGTTACTCTTAGCGTTCCTATCGGTACAGCCATCAAATCATGCCTATAAATCATGCCTATATTTGCGTGCAGTCAGTATCATCTACATGAAAAAAACTCCCGCAATTTCTTATAGAATACGTTGAAAATTAAATGTACGCGCCAAGATAAGATAACATATATCTAGATGCAGTAATATACACAGATTCCCGCGGACGTGGGAAGGAAAAAATTAGATAACAAAATCTGAGTGATATGGAAATTCCGCTGTATAGCTCATATCTTTCCCTACCTGCAGA'
CCM003['need_reverse']=0#if parts needs to be reversed:1 else:0
CCM003['Primer_up_U']='U3' #user site up
CCM003['Primer_down_U']='GV2'#user site down
CCM003['Primer_up']=''
CCM003['Primer_up_Tm']=''
CCM003['Primer_down']=''
CCM003['Primer_down_Tm']=''
pCCM016=[CCM009,CCM005,CCM006,CCM003]


In [15]:
CCM001={}
CCM001['name']='pCCM001(pCYC_benO:yEGFP:Cyc1t)'
CCM001['seq']='GATCCAGGCAACTTTAGTGCtgacacataatactccataggtattttattatacaaataatgtgtttgaacttattaaaacattcttttaaggtataaacaacaggcatatatatatgtgtgcgacgacacatgatcatatggcatgcatgtgctctgtatgtatataaaactcttgttttcttcttttctctaaatattctttccttatacattaggacctttgcagcataaattactatacttctatagacacacaaacacaaatacacacactaaattaataatctgtcataaaacaatgtctaaaggtgaagaattattcactggtgttgtcccaattttggttgaattagatggtgatgttaatggtcacaaattttctgtctccggtgaaggtgaaggtgatgctacttacggtaaattgaccttaaaatttatttgtactactggtaaattgccagttccatggccaaccttagtcactactttcggttatggtgttcaatgttttgcgagatacccagatcatatgaaacaacatgactttttcaagtctgccatgccagaaggttatgttcaagaaagaactatttttttcaaagatgacggtaactacaagaccagagctgaagtcaagtttgaaggtgataccttagttaatagaatcgaattaaaaggtattgattttaaagaagatggtaacattttaggtcacaaattggaatacaactataactctcacaatgtttacatcatggctgacaaacaaaagaatggtatcaaagttaacttcaaaattagacacaacattgaagatggttctgttcaattagctgaccattatcaacaaaatactccaattggtgatggtccagtcttgttaccagacaaccattacttatccactcaatctgccttatccaaagatccaaacgaaaagagagaccacatggtcttgttagaatttgttactgctgctggtattacccatggtatggatgaattgtacaaataaatcgcgtgcattcatccgctctaaccgaaaaggaaggagttagacaacctgaagtctaggtccctatttatttttttatagttatgttagtattaagaacgttatttatatttcaaatttttcttttttttctgtacagacgcgtgtacgcatgtaacattatactgaaaaccttgcttgagaaggttttgggacgctcgaAGATCGCGTCAGCTGAA'
CCM001['need_reverse']=0 #if parts needs to be reversed:1 else:0
CCM001['Primer_up_U']='GV1' #user site up
CCM001['Primer_down_U']='U1'#user site down
CCM001['Primer_up']=''
CCM001['Primer_up_Tm']=''
CCM001['Primer_down']=''
CCM001['Primer_down_Tm']=''

#this part also works for pCCM010
CCM002={}
CCM002['name']='pCCM002(tADH1::BenM::pRev1)'
CCM002['seq']='gcgacctcatgctatacctgagaaagcaacctgacctacaggaaagagttactcaagaataagaattttcgttttaaaacctaagagtcactttaaaatttgtatacacttattttttttataacttatttaataataaaaatcataaatcataagaaattcgcttatttagaagtgtcaacaacgtatctaccaacggaatgCGTGCGAtGAATGCGGCCGCTTtaccaatttggtggttcagtaaaaccttcgtaggcgtagatttgtctgatggtttcgtataatgagtagatgtaggtggattcttccatgtttctaacagcaatgtagattggggtaatagcatctggatctaataatgggacgtaggacaagttgaataattgaatggattgggtagaagctggaaccaatgaaataccttcaccagcagcaaccaaacccaaggccaattggacttctctaacttcgttaatcttggtaggttccaaaccatggtcagagaagatgttcataacatgagtagagaagtttggctttggagaagatgggtacaacaagatcttttcgtcgatcaaatcgttcaagtgaacacccttatccttcatttgattcaatggatgggaagcatgaacagcaaccatcaatctttcgtttctcaacaaggttctcttgatggctggatcagaaatcttcaatctaccaaaaccagcgtcaattctaccttctttcaaagcttcggtttgagccttagtacccatttcgtataattcgattctcaagtttggatgagcttgtctgtacaagtggataattcttggcaacaaaccgaacaacaaggaaccaacaaaaccgattctaatggtcttttcaacagaggcaattctcttggtcatactgaccatttgatcaacgttgcttagcaacttgatggcgtattggtaaaagaaatgaccttctggagtagtcttaactggtctggaacctctttccaataattggatacccaattcttcttccaagttttggatttgtctggataatggtggttgagcaatacacaacttgtcggcagcctttgtaaaagattgttcttcaacaacggcaacgaagtatctcaagtgtctcaattccattgttttatgacagatcgctggatatgcctagaaatgcagtatcgatttattttgagaaaatctgtcaactcgttacttctttgatagggaaatctttagttaaatttgtttaaaattaagaaccaggtctttagactatgccaaaaggtattggtgcttaaaggttgccgtaaactctaactctatatatctctaagccaatgtttaagttgatgcgatgtgcatgatcacttcacatttttttcatgattttgttttcttccttagctacttttttccgtcttggtctgtgtaaaaaaaaaatcgcttcactgaagaaaaatataaataaaaaaaggacatcagcaagttcgtagtcaataaatgtgatgagtaatttgcgataatctaccccaactcaaagcattcaattcaaatgaaagaagagttcgtgaattttaatctggcttacagattggatcatagcttctaatatatcttcctttttattttaaagtgattaaaatatgtggatgtctgcattaaggtgaaggcagactatagtgaaaaatagtagcaaaggcttcatataatgaactacctagttgtattcaattggaatctgtgaacgcggcattaaaagaatatttgagctttttttgattgtattaacctgaagctatcacggtttagactcattaaaagaaaaatatccggttgtatcggataacaacaccgctgtagtagcacagttttttgatcatagtttcattatttttttaaatatgcttgtatacagtaatctagttttggacggattattcatgtcagtaagattttactatctactttaaaattatcgaaacagatatcaactatacatccactcattattggcatagcgaaataaaatttatgaataggaaggatataaaacgaaaattgaaatcgtttagtattatatgaaactattagtcttttcgttttcgtcgtcatacttacattcttcttttataaatattgttgtgcctaagaagagctccagcttttgttccctttagtgagggttaattgcgcgcttggcgtaatcatggtcatagctgtttccTGTGTGAAATTGTTATCCGCT'
CCM002['need_reverse']=1 #if parts needs to be reversed:1 else:0
CCM002['Primer_up_U']='U1' #user site up
CCM002['Primer_down_U']='GV2'#user site down
CCM002['Primer_up']=''
CCM002['Primer_up_Tm']=''
CCM002['Primer_down']=''
CCM002['Primer_down_Tm']=''

pCCM017_18_19_20=[CCM001,CCM002]

In [16]:
#for mads: insertion of BTX in YER084W

YER084W_up={}
YER084W_up['name']='YER084W_CDS'
YER084W_up['seq']='ggtctttgcagcgctaggagcgtctaaaggtgattctgcgttcaaatgactgcttgcctgcccggtgattttattaagcctggagctggcacctccattgctaaacttcttctgtctcctttcccttagtaacctgcgcttttccgcctctgttaattcagacatatttcctagagctacgttgctgatgctacaaaca'
YER084W_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
YER084W_up['Primer_up_U']='GV1' #user site up
YER084W_up['Primer_down_U']='U1'#user site down
YER084W_up['Primer_up']=''
YER084W_up['Primer_up_Tm']=''
YER084W_up['Primer_down']=''
YER084W_up['Primer_down_Tm']=''

BTX={}
BTX['name']='ANTE113_BTX'
BTX['seq']='ATTAAGTCCTCAGCGAGCTCGCATGGAATGCGTGCGATGAGCGACCTCATGCTATACCTGAGAAAGCAACCTGACCTACAGGAAAGAGTTACTCAAGAATAAGAATTTTCGTTTTAAAACCTAAGAGTCACTTTAAAATTTGTATACACTTATTTTTTTTATAACTTATTTAATAATAAAAATCATAAATCATAAGAAATTCGCTTATTTAGAAGTGTCAACAACGTATCTACCAACGGAATGCGTGCGATTTACTCTTCCAACCTTCTTAGCAAGTATTCCACCTCAACTTCCTTAGTGATAGGTATAACAATTTCCTTATAAATTTTTACCAAATATTCTGGAGTAATCCTTCTTTCACCTGACTCGTTGGTAGGGTCCACACCATAGACTTCAGCCTTCTTAGTTAATCTTTCTAGAATCTTTTCTTCAACGGCAGAATTGGTGATATTCTTCATTATCCCCTCGACATCTTTACTTTTGATCAGCTTTGTGTATAGCGGGATATCCGATTGGAACTTGGCTTCAGCAACAAACTTGCCAAAGTGGATTCTCCTACTCAAGCTTTGCAAACATTCTATATCTCTAGTGGCAACAGAACTGAAGTTATTCTTATCATCACCATCTCTTTTCGAAATTAATGGTATAATCTTTTCAATATAAACTTTTTTTATTTTATCATTGTAATTAACTTCTGGGGCATAAGGCGCCAAAATTTGTGGGTAGTTAATGCTCGGTAAGAATGATTTCTGAATCTTGTCAGGAAAGAAGGGAGTTTCATCAGGTGATTCGAATCTTCTGATGCGAGAATGCGCAATTTCAAGATTTGAAAGAGCCCAATCCAAGAAAGATCCTTTAAAATTCGGAATTTCTAAACCTGGATGGTTTGCCTCATAAACTGAAGGACATGTGGCGAAATGCGACCTCTCAATAAATTTGAAGATGATCGAATCCTCCATTCTAACTAATTCATCTCTAATATTTTGTAGATTTAAAACAGTTTCTGGTTTTGTGAAATCCATagacttcgtCCGCTTGTAATTAAAACTTAGATTAGATTGCTATGCTTTCTTTCTAATGAGCAAGAAGTAAAAAAAGTTGTAATAGAACAAGAAAAATGAAACTGAAACTTGAGAAATTGAAGACCGTTTATTAACTTAAATATCAATGGGAGGTCATCGAAAGAGAAAAAAATCAAAAAAAAAAATTTTCAAGAAAAAGAAACGTGATAAAAATTTTTATTGCCTTTTTCGACGAAGAAAAAGAAACGAGGCGGTCTCTTTTTTCTTTTCCAAACCTTTAGTACGGGTAATTAACGACACCCTAGAGGAAGAAAGAGGGGAAATTTAGTATGCTGTGCTTGGGTGTTTTGAAGTGGTACGGCGATGCGCGGAGTCCGAGAAAATCTGGAAGAGTAAAAAAGGAGTAGAAACATTTTGAAGCTATGGTGTGTGCGGCCGGCCTGGAAGTACCTTCAAAGAATGGGGTCTTATCTTGTTTTGCAAGTACCACTGAGCAGGATAATAATAGAAATGATAATATACTATAGTAGAGATAACGTCGATGACTTCCCATACTGTAATTGCTTTTAGTTGTGTATTTTTAGTGTGCAAGTTTCTGTAAATCGATTAATTTTTTTTTCTTTCCTCTTTTTATTAACCTTAATTTTTATTTTAGATTCCTGACTTCAACTCAAGACGCACAGATATTATAACATCTGCATAATAGGCATTTGCAAGAATTACTCGTGAGTAAGGAAAGAGTGAGGAACTATCGCATACCTGCATTTAAAGATGCCGATTTGGGCGCGAATCCTTTATTTTGGCTTCACCCTCATACTATTATCAGGGCCAGAAAAAGGAAGTGTTTCCCTCCTTCTTGAATTGATGTTACCCTCATAAAGCACGTGGCCTCTTATCGAGAAAGAAATTACCGTCGCTCGTGATTTGTTTGCAAAAAGAACAAAACTGAAAAAACCCAGACACGCTCGACTTCCTGTCTTCCTATTGATTGCAGCTTCCAATTTCGTCACACAACAAGGTCCTAGCGACGGCTCACAGGTTTTGTAACAAGCAATCGAAGGTTCTGGAATGGCGGGAAAGGGTTTAGTACCACATGCTATGATGCCCACTGTGATCTCCAGAGCAAAGTTCGTTCGATCGTACTGTTACTCTCTCTCTTTCAAACAGAATTGTCCGAATCGTGTGACAACAACAGCCTGTTCTCACACACTCTTTTCTTCTAACCAAGGGGGTGGTTTAGTTTAGTAGAACCTCGTGAAACTTACATTTACATATATATAAACTTGCATAAATTGGTCAATGCAAGAAATACATATTTGGTCTTTTCTAATTCGTAGTTTTTCAAGTTCTTAGATGCTTTCTTTTTCTCTTTTTTACAGATCATCAAGGAAGTAATTATCTACTTTTTACAACAAATATAAAACAAGGATCCaagtctactATGAGTGAATCTCCAATGTTCGCTGCCAACGGCATGCCAAAGGTAAATCAAGGTGCTGAAGAAGATGTCAGAATTTTAGGTTACGACCCATTAGCTTCTCCAGCTCTCCTTCAAGTGCAAATCCCAGCCACACCAACTTCTTTGGAAACTGCCAAGAGAGGTAGAAGAGAAGCTATAGATATTATTACCGGTAAAGACGACAGAGTTCTTGTCATTGTCGGTCCTTGTTCCATCCATGATCTAGAAGCCGCTCAAGAATACGCTTTGAGATTAAAGAAATTGTCAGATGAATTAAAAGGTGATTTATCCATCATTATGAGAGCATACTTGGAGAAGCCAAGAACAACCGTCGGCTGGAAAGGTCTAATTAATGACCCTGATGTTAACAACACTTTCAACATCAACAAGGGTTTGCAATCCGCTAGACAATTGTTTGTCAACTTGACAAATATCGGTTTGCCAATTGGTTCTGAAATGCTTGATACCATTTCTCCTCAATACTTGGCTGATTTGGTCTCCTTCGGTGCCATTGGTGCCAGAACCACCGAATCTCAACTGCACAGAGAATTGGCCTCCGGTTTGTCTTTCCCAGTTGGTTTCAAGAACGGTACCGATGGTACCTTAAATGTTGCTGTGGATGCTTGTCAAGCCGCTGCTCATTCTCACCATTTCATGGGTGTTACTTTACATGGTGTTGCTGCTATCACCACTACTAAGGGTAACGAACACTGCTTCGTTATTCTAAGAGGTGGTAAAAAGGGTACCAACTACGACGCTAAGTCCGTTGCAGAAGCTAAGGCTCAATTGCCTGCCGGTTCCAACGGTCTAATGATTGACTACTCTCACGGTAACTCCAATAAGGATTTCAGAAACCAACCAAAGGTCAATGACGTTGTTTGTGAGCAAATCGCTAACGGTGAAAACGCCATTACCGGTGTCATGATTGAATCAAACATCAACGAAGGTAACCAAGGCATCCCAGCCGAAGGTAAAGCCGGCTTGAAATATGGTGTTTCCATCACTGATGCTTGTATAGGTTGGGAAACTACTGAAGACGTCTTGAGGAAATTGGCTGCTGCTGTCAGACAAAGAAGAGAAGTTAACAAGAAATAGATTCTGCAGGTAGGGAAAGATATGAGCTATACAGCGGAATTTCCATATCACTCAGATTTTGTTATCTAATTTTTTCCTTCCCACGTCCGCGGGAATCTGTGTATATTACTGCATCTAGATATATGTTATCTTATCTTGGCGCGTACATTTAATTTTCAACGTATTCTATAAGAAATTGCGGGAGTTTTTTTCATGTAGATGATACTGACTGCACGCAAATATAGGCATGATTTATAGGCATGATTTGATGGCTGTACCGATAGGAACGCTAAGAGTAACTTCAGAATCGTTATCCTGGCGGAAAAAATTCATTTGTAAACTTTAAAAAAAAAAGCCAATATCCCCAAAATTATTAAGAGCGCCTCCATTATTAACTAAAATTTCACTCAGCATCCACAATGTATCAGGTATCTACTACAGATATTACATGTGGCGAAAAAGACAAGAACAATGCAATAGCGCATCAAGAAAAAACACAAAGCTTTCAATCAATGAATCGAAAATGTCATTAAAATAGTATATAAATTGAAACTAAGTCATAAAGCTATAAAAAGAAAATTTATTTAAATGCAAGATTTAAAGTAAATTCACTTAGGATCCGTATCTTGGAATTGGGATCAATTTTAAAGGTTTAGTTTTCTGCAAGGCTATACCAAACTTTTCATCCATGTCCAAATCTTTAGGTGAAATGTCTCCCTCTAATTTCCAGTTGAAGAATTGCAATAATGTGGCCAACATCAATGTCAACATCCTTATGGCCAAATTCATACCAGGGCAAATTCTTCTACCAGCACCAAAAGGCAATAAACCAAAATCTCTTCCCTTTACATCAATCTCACAACCTATAAATCTTTCAGGTGAGAAAATATCGGCATTTTGCCAAGCATTTGGATCTCTACCTATGGCCCACAAGTTTACCAAAATTTGAGCATCCTTTGGAACAATGTATCCGTACAATTCTACGTCTGTGTCGGCTTTTCTAGGCAACAAGAAAACTGTAGGTGGGTGCAACCTCAAAGTTTCCTTAATGATTGCTTGCAAGTATGGCAAATTTATTATGTCTGACTCTTGGATTTGCTTATCCTTACCCAATACCTGTTTTATTTCCTCTTGGGCCTTTTCCATCATTTCAGGGTTTCTTATTAACTCAGTCATAACCCACTCAAAGGTAGATGATGTAGTATCAGTACCGGCATCGAAAATATCGACTAACAAGTGGTTAATCTCTCCCATAGTTAATTCATTCTGCTTAAACAATTGCAATAATACGTCCAAGACATCATCTGTAGTTGTAGTAGTTGTAGTTGATGAATCTGGAGCTAACCTCTCACATATTATTCCTTGAAATACTGCAATTAACTTATCAAAAGAACATGCCAACCTTCTTCTAATTCCTGATGGATCGACGCAACCTAAAATTGGGAAGTAATCGGCGTAGTTTGGTTTACCGATGTCTTCCATAATATTCCAAATTAATTCCTTAAACTCTTGAGAGGTATGTGATTTATGATGGGCCAATTCAACAGAAAAGAATAATTTAGACAATAAATTCAAAGAAGTTGTGAAGGCAGCTTTACCAATGTCAACTGCCTGACCCTTCTGAGCGCACTCTTGAACGTATTCATATAATTGCTGAACCTTTGCATGTCTAAAGGTCTGGCATGCATCTAATCTTTGTGGAGACAATAAATGAACAGCAGTTATCTTTCTGAAGTTTCTCCATTTAGGTGAAACAGGCAACCAAGACATAGTCAATTTGTGGTGGTCACCAGCAGTTACTGAATTTGGAATTGTCCTGTTTGATAAAGGGTGATCCTTCTTCAAGAACATCTCTTTGGCGACATCAGCAGATGATACAACAATGGTGGTAACTGATCCTAACCTCAAAGAGATTAAAGGACCGTGGATCTTAGCCAAATTTGCAAATGATCTATGAGGCTTCTTTCCTACTTCCAAGATATTACCAATAATTGGTAAAGGTTTAGGACCTGGAGGTAACAATTTTGTTGTTTGTTGTGAGAATAATAATTTAATAAAATGAAATGAAATAAACCAAATTGCTAATATCATAGCTAATGTTGCATGATCCATTTTTAGTTTATGTATGTGTTTTTTGTAGTTATAGATTTAAGCAAGAAAAGAATACAAACAAAAAATTGAAAAAGATTGATTTAGAATTAAAAAGAAAAATATTTACGTAAGAAGGGAAAATAGTAAATGTTGCAAGTTCACTAAACTCCTAAATTATGCTGCCCTTTATATTCCCTGTTACAGCAGCCGAGCCAAAGGTATATAGGCTCCTTTGCATTAGCATGCGTAACAAACCACCTGTCAGTTTCAACCGAGGTGGTATCCGAGAGAATTGTGTGATTGCTTTAATTAATTTCGGAGAATCTCACATGCCACTGAAGATTAAAAACTGGATGCCAGAAAAGGGGTGTCCAGGTGTAACATCAATAGAGGAAGCTGAAAAGTCTTAGAACGGGTAATCTTCCACCAACCTGATGGGTTCCTAGATATAATCTCGAAGGGAATAAGTAGGGTGATACCGCAGAAGTGTCTGAATGTATTAAGGTCCTCACAGTTTAAATCCCGCTCACACTAACGTAGGATTATTATAACTCAAAAAAATGGCATTATTCTAAGTAAGTTAAATATCCGTAATCTTTAAACAagacttcgTCACCCATGAACCACACGGTTAGTCCAAAAGGGGCAGTTCAGATTCCAGATGCGGGAATTAGCTTGCTGCCACCCTCACCTCACTAACGCTGCGGTGTGCGGATACTTCATGCTATTTATAGACGCGCGTGTCGGAATCAGCACGCGCAAGAACCAAATGGGAAAATCGGAATGGGTCCAGAACTGCTTTGAGTGCTGGCTATTGGCGTCTGATTTCCGTTTTGGGAATCCTTTGCCGCGCGCCCCTCTCAAAACTCCGCACAAGTCCCAGAAAGCGGGAAAGAAATAAAACGCCACCAAAAAAAAAAAAATAAAAGCCAATCCTCGAAGCGTGGGTGGTAGGCCCTGGATTATCCCGTACAAGTATTTCTCAGGAGTAAAAAAACCGTTTGTTTTGGAATTTCCCATTTCGCGGCCACCTACGCCGCTATCTTTGCAACAACTATCTGCGATAACTCAGCAAATTTTGCATATTCGTGTTGCAGTATTGCGATAATGGGAGTCTTACTTCCAACATAACGGCAGAAAGAAATGTGAGAAAATTTTGCATCCTTTGCCTCCGTTCAAGTATATAAAGTCGGCATGCTTGATAATCTTTCTTTCCATCCTACATTGTTCTAATTATTCTTATTCTCCTTTATTCTTTCCTAACATACCAAGAAATTAATCTTCTGTCATTCGCTTAAACACTATATCAATAAAGATCTAAAAATGAAGGGAACCTACTACATCAACCACGGTGACCCATTGATGTACTTGAAGAAGCACATCAAGTTGAGACAATTCTTGGAAGGTTGGCAAGAAAACGTTGTTATCGAAAAGCCAAAGTCTATCTTGATCATCAGTGCACACTGGGACACCAACGTTCCAACCGTTAACTTCGTTGAACACTGTGACACCATCCACGACTTCGACGACTACCCAGACCCATTGTACCAAATCCAATACCGCGCTCCAGGTGCTCCAAACTTGGCTAAGAAGGTTGAAGAATTGCTTAAGGAATCTGGTATGGAATGTGAAATCGACACCAAGAGAGGTTTGGACCACGCTGCTTGGTTCCCATTGATGTTCATGTACCCAGAAGCTAACATCCCAATCTGTGAATTGTCTGTTCAACCATCTAAGGACGGTATCCACCACTACAACGTTGGTAAAGCTTTGTCTCCATTGTTGCAACAAGGTGTTTTGATCATCGGTTCTGGTGGAACCGTTCACCCATCTGACGACACCCCACACTGTCCAAACGGTGTTGCTCCATGGGCTATCGAGTTCGACAACTGGTTGGAAGACGCTTTGCTCTCGGGTAGATACGAAGACGTTAACAACTTCAAGAAGTTGGCTCCAAACTGGGAAATCTCTCACCCAGGTCAAGAACACTTGTACCCATTGCACGTTGCTTTGGGTGCTGCTGGTAAGAACCCAAAGACCCAATTGATCCACCGGTCTTGGGCTGCTAACGGTGTTTTCGGTTACTCTACCTACAACTTCACCCCAACCACCCAAAAGACCGACGGATCCTAAATCGCGTGCATTCATCCGCTCTAACCGAAAAGGAAGGAGTTAGACAACCTGAAGTCTAGGTCCCTATTTATTTTTTTATAGTTATGTTAGTATTAAGAACGTTATTTATATTTCAAATTTTTCTTTTTTTTCTGTACAGACGCGTGTACGCATGTAACATTATACTGAAAACCTTGCTTGAGAAGGTTTTGGGACGCTCGAAGATCGCGTCAGCTGAAGCTTCGTACGCTGCAGGTCGACAATTCTGCAGGTCGACAACCCTTAATATAACTTCGTATAATGTATGCTATACGAAGTTATTAGGTCTAGAGATCTGTTTAGCTTGCCTCGTCCCCGCCGGGTCACCCGGCCAGCGACATGGAGGCCCAGAATACCCTCCTTGACAGTCTTGACGTGCGCAGCTCAGGGGCATGATGTGACTGTCGCCCGTACATTTAGCCCATACATCCCCATGTATAATCATTTGCATCCATACATTTTGATGGCCGCACGGCGCGAAGCAAAAATTACGGCTCCTCGCTGCAGACCTGCGAGCAGGGAAACGCTCCCCTCACAGACGCGTTGAATTGTCCCCACGCCGCGCCCCTGTAGAGAAATATAAAAGGTTAGGATTTGCCACTGAGGTTCTTCTTTCATATACTTCCTTTTAAAATCTTGCTAGGATACAGTTCTCACATCACATCCGAACATAAACAACCATGGGTACCACTCTTGACGACACGGCTTACCGGTACCGCACCAGTGTCCCGGGGGACGCCGAGGCCATCGAGGCACTGGATGGGTCCTTCACCACCGACACCGTCTTCCGCGTCACCGCCACCGGGGACGGCTTCACCCTGCGGGAGGTGCCGGTGGACCCGCCCCTGACCAAGGTGTTCCCCGACGACGAATCGGACGACGAATCGGACGACGGGGAGGACGGCGACCCGGACTCCCGGACGTTCGTCGCGTACGGGGACGACGGCGACCTGGCGGGCTTCGTGGTCGTCTCGTACTCCGGCTGGAACCGCCGGCTGACCGTCGAGGACATCGAGGTCGCCCCGGAGCACCGGGGGCACGGGGTCGGGCGCGCGTTGATGGGGCTCGCGACGGAGTTCGCCCGCGAGCGGGGCGCCGGGCACCTCTGGCTGGAGGTCACCAACGTCAACGCACCGGCGATCCACGCGTACCGGCGGATGGGGTTCACCCTCTGCGGCCTGGACACCGCCCTGTACGACGGCACCGCCTCGGACGGCGAGCAGGCGCTCTACATGAGCATGCCCTGCCCCTAATCAGTACTGACAATAAAAAGATTCTTGTTTTCAAGAACTTGTCATTTGTATAGTTTTTTTATATTGTAGTTGTTCTATTTTAATCAAATGTTAGCGTGATTTATATTTTTTTTCGCCTCGACATCATCTGCCCAGATGCGAAGTTAAGTGCGCAGAAAGTAATATCATGCGTCAATCGTATGTGAATGCTGGTCGCTATACTGCTGTCGATTCGATACTAACGCCGCCATCCAGTGTCGAAAACGAGCTCTCGAGAACCCTTAATATAACTTCGTATAATGTATGCTATACGAAGTTATTAGGTGATATCAGATCCACTAGTG'
BTX['need_reverse']=0 #if parts needs to be reversed:1 else:0
BTX['Primer_up_U']='U1' #user site up
BTX['Primer_down_U']='U2'#user site down
BTX['Primer_up']=''
BTX['Primer_up_Tm']=''
BTX['Primer_down']=''
BTX['Primer_down_Tm']=''

YER084W_dw={}
YER084W_dw['name']='YER084W_CDS'
YER084W_dw['seq']='taactcagattttcaagaaacagaatagcacattctctaacatcaagttatcataccaatatataacacaaattatgcataaatagataagtaatatgtccctctgcccctgcctttccctttccagatgctggcatcgaaagtaaggcatgtatttgtatgtaagttttttaattaatagtcttcttctttcttatcag'
YER084W_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
YER084W_dw['Primer_up_U']='U2' #user site up
YER084W_dw['Primer_down_U']='GV2'#user site down
YER084W_dw['Primer_up']=''
YER084W_dw['Primer_up_Tm']=''
YER084W_dw['Primer_down']=''
YER084W_dw['Primer_down_Tm']=''

YER084W_KO_BTX=[YER084W_up,BTX,YER084W_dw]

In [9]:
DOPA1={}
DOPA1['name']='ANTE113:ARO7G141S_ARO4K229L_vCYP76AD1'
DOPA1['seq']='GCGACCTCATGCTATACCTGAGAAAGCAACCTGACCTACAGGAAAGAGTTACTCAAGAATAAGAATTTTCGTTTTAAAACCTAAGAGTCACTTTAAAATTTGTATACACTTATTTTTTTTATAACTTATTTAATAATAAAAATCATAAATCATAAGAAATTCGCTTATTTAGAAGTGTCAACAACGTATCTACCAACGGAATGCGTGCGATTTACTCTTCCAACCTTCTTAGCAAGTATTCCACCTCAACTTCCTTAGTGATAGGTATAACAATTTCCTTATAAATTTTTACCAAATATTCTGGAGTAATCCTTCTTTCACCTGACTCGTTGGTAGGGTCCACACCATAGACTTCAGCCTTCTTAGTTAATCTTTCTAGAATCTTTTCTTCAACGGCAGAATTGGTGATATTCTTCATTATCCCCTCGACATCTTTACTTTTGATCAGCTTTGTGTATAGCGGGATATCCGATTGGAACTTGGCTTCAGCAACAAACTTGCCAAAGTGGATTCTCCTACTCAAGCTTTGCAAACATTCTATATCTCTAGTGGCAACAGAACTGAAGTTATTCTTATCATCACCATCTCTTTTCGAAATTAATGGTATAATCTTTTCAATATAAACTTTTTTTATTTTATCATTGTAATTAACTTCTGGGGCATAAGGCGCCAAAATTTGTGGGTAGTTAATGCTCGGTAAGAATGATTTCTGAATCTTGTCAGGAAAGAAGGGAGTTTCATCAGGTGATTCGAATCTTCTGATGCGAGAATGCGCAATTTCAAGATTTGAAAGAGCCCAATCCAAGAAAGATCCTTTAAAATTCGGAATTTCTAAACCTGGATGGTTTGCCTCATAAACTGAAGGACATGTGGCGAAATGCGACCTCTCAATAAATTTGAAGATGATCGAATCCTCCATTCTAACTAATTCATCTCTAATATTTTGTAGATTTAAAACAGTTTCTGGTTTTGTGAAATCCATagacttcgtCCGCTTGTAATTAAAACTTAGATTAGATTGCTATGCTTTCTTTCTAATGAGCAAGAAGTAAAAAAAGTTGTAATAGAACAAGAAAAATGAAACTGAAACTTGAGAAATTGAAGACCGTTTATTAACTTAAATATCAATGGGAGGTCATCGAAAGAGAAAAAAATCAAAAAAAAAAATTTTCAAGAAAAAGAAACGTGATAAAAATTTTTATTGCCTTTTTCGACGAAGAAAAAGAAACGAGGCGGTCTCTTTTTTCTTTTCCAAACCTTTAGTACGGGTAATTAACGACACCCTAGAGGAAGAAAGAGGGGAAATTTAGTATGCTGTGCTTGGGTGTTTTGAAGTGGTACGGCGATGCGCGGAGTCCGAGAAAATCTGGAAGAGTAAAAAAGGAGTAGAAACATTTTGAAGCTATGGTGTGTGCGGCCGGCCTGGAAGTACCTTCAAAGAATGGGGTCTTATCTTGTTTTGCAAGTACCACTGAGCAGGATAATAATAGAAATGATAATATACTATAGTAGAGATAACGTCGATGACTTCCCATACTGTAATTGCTTTTAGTTGTGTATTTTTAGTGTGCAAGTTTCTGTAAATCGATTAATTTTTTTTTCTTTCCTCTTTTTATTAACCTTAATTTTTATTTTAGATTCCTGACTTCAACTCAAGACGCACAGATATTATAACATCTGCATAATAGGCATTTGCAAGAATTACTCGTGAGTAAGGAAAGAGTGAGGAACTATCGCATACCTGCATTTAAAGATGCCGATTTGGGCGCGAATCCTTTATTTTGGCTTCACCCTCATACTATTATCAGGGCCAGAAAAAGGAAGTGTTTCCCTCCTTCTTGAATTGATGTTACCCTCATAAAGCACGTGGCCTCTTATCGAGAAAGAAATTACCGTCGCTCGTGATTTGTTTGCAAAAAGAACAAAACTGAAAAAACCCAGACACGCTCGACTTCCTGTCTTCCTATTGATTGCAGCTTCCAATTTCGTCACACAACAAGGTCCTAGCGACGGCTCACAGGTTTTGTAACAAGCAATCGAAGGTTCTGGAATGGCGGGAAAGGGTTTAGTACCACATGCTATGATGCCCACTGTGATCTCCAGAGCAAAGTTCGTTCGATCGTACTGTTACTCTCTCTCTTTCAAACAGAATTGTCCGAATCGTGTGACAACAACAGCCTGTTCTCACACACTCTTTTCTTCTAACCAAGGGGGTGGTTTAGTTTAGTAGAACCTCGTGAAACTTACATTTACATATATATAAACTTGCATAAATTGGTCAATGCAAGAAATACATATTTGGTCTTTTCTAATTCGTAGTTTTTCAAGTTCTTAGATGCTTTCTTTTTCTCTTTTTTACAGATCATCAAGGAAGTAATTATCTACTTTTTACAACAAATATAAAACAAGGATCCaagtctactATGAGTGAATCTCCAATGTTCGCTGCCAACGGCATGCCAAAGGTAAATCAAGGTGCTGAAGAAGATGTCAGAATTTTAGGTTACGACCCATTAGCTTCTCCAGCTCTCCTTCAAGTGCAAATCCCAGCCACACCAACTTCTTTGGAAACTGCCAAGAGAGGTAGAAGAGAAGCTATAGATATTATTACCGGTAAAGACGACAGAGTTCTTGTCATTGTCGGTCCTTGTTCCATCCATGATCTAGAAGCCGCTCAAGAATACGCTTTGAGATTAAAGAAATTGTCAGATGAATTAAAAGGTGATTTATCCATCATTATGAGAGCATACTTGGAGAAGCCAAGAACAACCGTCGGCTGGAAAGGTCTAATTAATGACCCTGATGTTAACAACACTTTCAACATCAACAAGGGTTTGCAATCCGCTAGACAATTGTTTGTCAACTTGACAAATATCGGTTTGCCAATTGGTTCTGAAATGCTTGATACCATTTCTCCTCAATACTTGGCTGATTTGGTCTCCTTCGGTGCCATTGGTGCCAGAACCACCGAATCTCAACTGCACAGAGAATTGGCCTCCGGTTTGTCTTTCCCAGTTGGTTTCAAGAACGGTACCGATGGTACCTTAAATGTTGCTGTGGATGCTTGTCAAGCCGCTGCTCATTCTCACCATTTCATGGGTGTTACTTTACATGGTGTTGCTGCTATCACCACTACTAAGGGTAACGAACACTGCTTCGTTATTCTAAGAGGTGGTAAAAAGGGTACCAACTACGACGCTAAGTCCGTTGCAGAAGCTAAGGCTCAATTGCCTGCCGGTTCCAACGGTCTAATGATTGACTACTCTCACGGTAACTCCAATAAGGATTTCAGAAACCAACCAAAGGTCAATGACGTTGTTTGTGAGCAAATCGCTAACGGTGAAAACGCCATTACCGGTGTCATGATTGAATCAAACATCAACGAAGGTAACCAAGGCATCCCAGCCGAAGGTAAAGCCGGCTTGAAATATGGTGTTTCCATCACTGATGCTTGTATAGGTTGGGAAACTACTGAAGACGTCTTGAGGAAATTGGCTGCTGCTGTCAGACAAAGAAGAGAAGTTAACAAGAAATAGATTCTGCAGGTAGGGAAAGATATGAGCTATACAGCGGAATTTCCATATCACTCAGATTTTGTTATCTAATTTTTTCCTTCCCACGTCCGCGGGAATCTGTGTATATTACTGCATCTAGATATATGTTATCTTATCTTGGCGCGTACATTTAATTTTCAACGTATTCTATAAGAAATTGCGGGAGTTTTTTTCATGTAGATGATACTGACTGCACGCAAATATAGGCATGATTTATAGGCATGATTTGATGGCTGTACCGATAGGAACGCTAAGAGTAACTTCAGAATCGTTATCCTGGCGGAAAAAATTCATTTGTAAACTTTAAAAAAAAAAGCCAATATCCCCAAAATTATTAAGAGCGCCTCCATTATTAACTAAAATTTCACTCAGCATCCACAATGTATCAGGTATCTACTACAGATATTACATGTGGCGAAAAAGACAAGAACAATGCAATAGCGCATCAAGAAAAAACACAAAGCTTTCAATCAATGAATCGAAAATGTCATTAAAATAGTATATAAATTGAAACTAAGTCATAAAGCTATAAAAAGAAAATTTATTTAAATGCAAGATTTAAAGTAAATTCACTTAGGATCCGTATCTTGGAATTGGGATCAATTTTAAAGGTTTAGTTTTCTGCAAGGCTATACCAAACTTTTCATCCATGTCCAAATCTTTAGGTGAAATGTCTCCCTCTAATTTCCAGTTGAAGAATTGCAATAATGTGGCCAACATCAATGTCAACATCCTTATGGCCAAATTCATACCAGGGCAAATTCTTCTACCAGCACCAAAAGGCAATAAACCAAAATCTCTTCCCTTTACATCAATCTCACAACCTATAAATCTTTCAGGTGAGAAAATATCGGCATTTTGCCAAGCATTTGGATCTCTACCTATGGCCCACAAGTTTACCAAAATTTGAGCATCCTTTGGAACAATGTATCCGTACAATTCTACGTCTGTGTCGGCTTTTCTAGGCAACAAGAAAACTGTAGGTGGGTGCAACCTCAAAGTTTCCTTAATGATTGCTTGCAAGTATGGCAAATTTATTATGTCTGACTCTTGGATTTGCTTATCCTTACCCAATACCTGTTTTATTTCCTCTTGGGCCTTTTCCATCATTTCAGGGTTTCTTATTAACTCAGTCATAACCCACTCAAAGGTAGATGATGTAGTATCAGTACCGGCATCGAAAATATCGACTAACAAGTGGTTAATCTCTCCCATAGTTAATTCATTCTGCTTAAACAATTGCAATAATACGTCCAAGACATCATCTGTAGTTGTAGTAGTTGTAGTTGATGAATCTGGAGCTAACCTCTCACATATTATTCCTTGAAATACTGCAATTAACTTATCAAAAGAACATGCCAACCTTCTTCTAATTCCTGATGGATCGACGCAACCTAAAATTGGGAAGTAATCGGCGTAGTTTGGTTTACCGATGTCTTCCATAATATTCCAAATTAATTCCTTAAACTCTTGAGAGGTATGTGATTTATGATGGGCCAATTCAACAGAAAAGAATAATTTAGACAATAAATTCAAAGAAGTTGTGAAGGCAGCTTTACCAATGTCAACTGCCTGACCCTTCTGAGCGCACTCTTGAACGTATTCATATAATTGCTGAACCTTTGCATGTCTAAAGGTCTGGCATGCATCTAATCTTTGTGGAGACAATAAATGAACAGCAGTTATCTTTCTGAAGTTTCTCCATTTAGGTGAAACAGGCAACCAAGACATAGTCAATTTGTGGTGGTCACCAGCAGTTACTGAATTTGGAATTGTCCTGTTTGATAAAGGGTGATCCTTCTTCAAGAACATCTCTTTGGCGACATCAGCAGATGATACAACAATGGTGGTAACTGATCCTAACCTCAAAGAGATTAAAGGACCGTGGATCTTAGCCAAATTTGCAAATGATCTATGAGGCTTCTTTCCTACTTCCAAGATATTACCAATAATTGGTAAAGGTTTAGGACCTGGAGGTAACAATTTTGTTGTTTGTTGTGAGAATAATAATTTAATAAAATGAAATGAAATAAACCAAATTGCTAATATCATAGCTAATGTTGCATGATCCATTTTTAGTTTATGTATGTGTTTTTTGTAGTTATAGATTTAAGCAAGAAAAGAATACAAACAAAAAATTGAAAAAGATTGATTTAGAATTAAAAAGAAAAATATTTACGTAAGAAGGGAAAATAGTAAATGTTGCAAGTTCACTAAACTCCTAAATTATGCTGCCCTTTATATTCCCTGTTACAGCAGCCGAGCCAAAGGTATATAGGCTCCTTTGCATTAGCATGCGTAACAAACCACCTGTCAGTTTCAACCGAGGTGGTATCCGAGAGAATTGTGTGATTGCTTTAATTAATTTCGGAGAATCTCACATGCCACTGAAGATTAAAAACTGGATGCCAGAAAAGGGGTGTCCAGGTGTAACATCAATAGAGGAAGCTGAAAAGTCTTAGAACGGGTAATCTTCCACCAACCTGATGGGTTCCTAGATATAATCTCGAAGGGAATAAGTAGGGTGATACCGCAGAAGTGTCTGAATGTATTAAGGTCCTCACAGTTTAAATCCCGCTCACACTAACGTAGGATTATTATAACTCAAAAAAATGGCATTATTCTAAGTAAGTTAAATATCCGTAATCTTTAAACA'
DOPA1['need_reverse']=0 #if parts needs to be reversed:1 else:0
DOPA1['Primer_up_U']='GV1' #user site up
DOPA1['Primer_down_U']='U1'#user site down
DOPA1['Primer_up']=''
DOPA1['Primer_up_Tm']=''
DOPA1['Primer_down']=''
DOPA1['Primer_down_Tm']=''

DOPA2={}
DOPA2['name']='ANTE113:NatMX'
DOPA2['seq']='TAGGTCTAGAGATCTGTTTAGCTTGCCTCGTCCCCGCCGGGTCACCCGGCCAGCGACATGGAGGCCCAGAATACCCTCCTTGACAGTCTTGACGTGCGCAGCTCAGGGGCATGATGTGACTGTCGCCCGTACATTTAGCCCATACATCCCCATGTATAATCATTTGCATCCATACATTTTGATGGCCGCACGGCGCGAAGCAAAAATTACGGCTCCTCGCTGCAGACCTGCGAGCAGGGAAACGCTCCCCTCACAGACGCGTTGAATTGTCCCCACGCCGCGCCCCTGTAGAGAAATATAAAAGGTTAGGATTTGCCACTGAGGTTCTTCTTTCATATACTTCCTTTTAAAATCTTGCTAGGATACAGTTCTCACATCACATCCGAACATAAACAACCATGGGTACCACTCTTGACGACACGGCTTACCGGTACCGCACCAGTGTCCCGGGGGACGCCGAGGCCATCGAGGCACTGGATGGGTCCTTCACCACCGACACCGTCTTCCGCGTCACCGCCACCGGGGACGGCTTCACCCTGCGGGAGGTGCCGGTGGACCCGCCCCTGACCAAGGTGTTCCCCGACGACGAATCGGACGACGAATCGGACGACGGGGAGGACGGCGACCCGGACTCCCGGACGTTCGTCGCGTACGGGGACGACGGCGACCTGGCGGGCTTCGTGGTCGTCTCGTACTCCGGCTGGAACCGCCGGCTGACCGTCGAGGACATCGAGGTCGCCCCGGAGCACCGGGGGCACGGGGTCGGGCGCGCGTTGATGGGGCTCGCGACGGAGTTCGCCCGCGAGCGGGGCGCCGGGCACCTCTGGCTGGAGGTCACCAACGTCAACGCACCGGCGATCCACGCGTACCGGCGGATGGGGTTCACCCTCTGCGGCCTGGACACCGCCCTGTACGACGGCACCGCCTCGGACGGCGAGCAGGCGCTCTACATGAGCATGCCCTGCCCCTAATCAGTACTGACAATAAAAAGATTCTTGTTTTCAAGAACTTGTCATTTGTATAGTTTTTTTATATTGTAGTTGTTCTATTTTAATCAAATGTTAGCGTGATTTATATTTTTTTTCGCCTCGACATCATCTGCCCAGATGCGAAGTTAAGTGCGCAGAAAGTAATATCATGCGTCAATCGTATGTGAATGCTGGTCGCTATACTGCTGTCGATTCGATACTAACGCCGCCATCCAGTGTCGAAAACGAGCTCTCGAGAACCCTTAAT'
DOPA2['need_reverse']=0 #if parts needs to be reversed:1 else:0
DOPA2['Primer_up_U']='U1' #user site up
DOPA2['Primer_down_U']='GV2'#user site down
DOPA2['Primer_up']=''
DOPA2['Primer_up_Tm']=''
DOPA2['Primer_down']=''
DOPA2['Primer_down_Tm']=''

DOPAs=[DOPA1,DOPA2]

In [69]:
### Design the primers for addind CDSs to the BTX pathway + making the KO:
###up regions (promoter): 1000bp upstream of CDS unless another sequeunce is there
### upstream is 500bp regardless of presence of other CDS (might include the end of another gene)
### for PDX3 promoter is close to snoRNA, so only took 240bp long promoter
### for PDX3 terminator is close to start of other CDS so took a shorter terminator
##down region (terminator): 500dp dwnstream of CDS unless another sequeunce is there

#for the KO primers (reverse of up and forward of down) to avoid nesting promoter>>promoter::KANMX::terminator>>terminator
#end up region so that total length is 200bp
#start dwn region so that total length is 200bp

#YDL119C =HEM25
#KGD2 v
#PPA1 = VMA16
#CAX4
#Pro2
#GLR1
#YSY6
#PDX3

#BTX pathway will be annealed straight into pCCM021 to be flanked by XII-up and  HomoB
BTX={}
BTX['name']='ANTE113_BTX'
BTX['seq']='GCGACCTCATGCTATACCTGAGAAAGCAACCTGACCTACAGGAAAGAGTTACTCAAGAATAAGAATTTTCGTTTTAAAACCTAAGAGTCACTTTAAAATTTGTATACACTTATTTTTTTTATAACTTATTTAATAATAAAAATCATAAATCATAAGAAATTCGCTTATTTAGAAGTGTCAACAACGTATCTACCAACGGAATGCGTGCGATTTACTCTTCCAACCTTCTTAGCAAGTATTCCACCTCAACTTCCTTAGTGATAGGTATAACAATTTCCTTATAAATTTTTACCAAATATTCTGGAGTAATCCTTCTTTCACCTGACTCGTTGGTAGGGTCCACACCATAGACTTCAGCCTTCTTAGTTAATCTTTCTAGAATCTTTTCTTCAACGGCAGAATTGGTGATATTCTTCATTATCCCCTCGACATCTTTACTTTTGATCAGCTTTGTGTATAGCGGGATATCCGATTGGAACTTGGCTTCAGCAACAAACTTGCCAAAGTGGATTCTCCTACTCAAGCTTTGCAAACATTCTATATCTCTAGTGGCAACAGAACTGAAGTTATTCTTATCATCACCATCTCTTTTCGAAATTAATGGTATAATCTTTTCAATATAAACTTTTTTTATTTTATCATTGTAATTAACTTCTGGGGCATAAGGCGCCAAAATTTGTGGGTAGTTAATGCTCGGTAAGAATGATTTCTGAATCTTGTCAGGAAAGAAGGGAGTTTCATCAGGTGATTCGAATCTTCTGATGCGAGAATGCGCAATTTCAAGATTTGAAAGAGCCCAATCCAAGAAAGATCCTTTAAAATTCGGAATTTCTAAACCTGGATGGTTTGCCTCATAAACTGAAGGACATGTGGCGAAATGCGACCTCTCAATAAATTTGAAGATGATCGAATCCTCCATTCTAACTAATTCATCTCTAATATTTTGTAGATTTAAAACAGTTTCTGGTTTTGTGAAATCCATagacttcgtCCGCTTGTAATTAAAACTTAGATTAGATTGCTATGCTTTCTTTCTAATGAGCAAGAAGTAAAAAAAGTTGTAATAGAACAAGAAAAATGAAACTGAAACTTGAGAAATTGAAGACCGTTTATTAACTTAAATATCAATGGGAGGTCATCGAAAGAGAAAAAAATCAAAAAAAAAAATTTTCAAGAAAAAGAAACGTGATAAAAATTTTTATTGCCTTTTTCGACGAAGAAAAAGAAACGAGGCGGTCTCTTTTTTCTTTTCCAAACCTTTAGTACGGGTAATTAACGACACCCTAGAGGAAGAAAGAGGGGAAATTTAGTATGCTGTGCTTGGGTGTTTTGAAGTGGTACGGCGATGCGCGGAGTCCGAGAAAATCTGGAAGAGTAAAAAAGGAGTAGAAACATTTTGAAGCTATGGTGTGTGCGGCCGGCCTGGAAGTACCTTCAAAGAATGGGGTCTTATCTTGTTTTGCAAGTACCACTGAGCAGGATAATAATAGAAATGATAATATACTATAGTAGAGATAACGTCGATGACTTCCCATACTGTAATTGCTTTTAGTTGTGTATTTTTAGTGTGCAAGTTTCTGTAAATCGATTAATTTTTTTTTCTTTCCTCTTTTTATTAACCTTAATTTTTATTTTAGATTCCTGACTTCAACTCAAGACGCACAGATATTATAACATCTGCATAATAGGCATTTGCAAGAATTACTCGTGAGTAAGGAAAGAGTGAGGAACTATCGCATACCTGCATTTAAAGATGCCGATTTGGGCGCGAATCCTTTATTTTGGCTTCACCCTCATACTATTATCAGGGCCAGAAAAAGGAAGTGTTTCCCTCCTTCTTGAATTGATGTTACCCTCATAAAGCACGTGGCCTCTTATCGAGAAAGAAATTACCGTCGCTCGTGATTTGTTTGCAAAAAGAACAAAACTGAAAAAACCCAGACACGCTCGACTTCCTGTCTTCCTATTGATTGCAGCTTCCAATTTCGTCACACAACAAGGTCCTAGCGACGGCTCACAGGTTTTGTAACAAGCAATCGAAGGTTCTGGAATGGCGGGAAAGGGTTTAGTACCACATGCTATGATGCCCACTGTGATCTCCAGAGCAAAGTTCGTTCGATCGTACTGTTACTCTCTCTCTTTCAAACAGAATTGTCCGAATCGTGTGACAACAACAGCCTGTTCTCACACACTCTTTTCTTCTAACCAAGGGGGTGGTTTAGTTTAGTAGAACCTCGTGAAACTTACATTTACATATATATAAACTTGCATAAATTGGTCAATGCAAGAAATACATATTTGGTCTTTTCTAATTCGTAGTTTTTCAAGTTCTTAGATGCTTTCTTTTTCTCTTTTTTACAGATCATCAAGGAAGTAATTATCTACTTTTTACAACAAATATAAAACAAGGATCCaagtctactATGAGTGAATCTCCAATGTTCGCTGCCAACGGCATGCCAAAGGTAAATCAAGGTGCTGAAGAAGATGTCAGAATTTTAGGTTACGACCCATTAGCTTCTCCAGCTCTCCTTCAAGTGCAAATCCCAGCCACACCAACTTCTTTGGAAACTGCCAAGAGAGGTAGAAGAGAAGCTATAGATATTATTACCGGTAAAGACGACAGAGTTCTTGTCATTGTCGGTCCTTGTTCCATCCATGATCTAGAAGCCGCTCAAGAATACGCTTTGAGATTAAAGAAATTGTCAGATGAATTAAAAGGTGATTTATCCATCATTATGAGAGCATACTTGGAGAAGCCAAGAACAACCGTCGGCTGGAAAGGTCTAATTAATGACCCTGATGTTAACAACACTTTCAACATCAACAAGGGTTTGCAATCCGCTAGACAATTGTTTGTCAACTTGACAAATATCGGTTTGCCAATTGGTTCTGAAATGCTTGATACCATTTCTCCTCAATACTTGGCTGATTTGGTCTCCTTCGGTGCCATTGGTGCCAGAACCACCGAATCTCAACTGCACAGAGAATTGGCCTCCGGTTTGTCTTTCCCAGTTGGTTTCAAGAACGGTACCGATGGTACCTTAAATGTTGCTGTGGATGCTTGTCAAGCCGCTGCTCATTCTCACCATTTCATGGGTGTTACTTTACATGGTGTTGCTGCTATCACCACTACTAAGGGTAACGAACACTGCTTCGTTATTCTAAGAGGTGGTAAAAAGGGTACCAACTACGACGCTAAGTCCGTTGCAGAAGCTAAGGCTCAATTGCCTGCCGGTTCCAACGGTCTAATGATTGACTACTCTCACGGTAACTCCAATAAGGATTTCAGAAACCAACCAAAGGTCAATGACGTTGTTTGTGAGCAAATCGCTAACGGTGAAAACGCCATTACCGGTGTCATGATTGAATCAAACATCAACGAAGGTAACCAAGGCATCCCAGCCGAAGGTAAAGCCGGCTTGAAATATGGTGTTTCCATCACTGATGCTTGTATAGGTTGGGAAACTACTGAAGACGTCTTGAGGAAATTGGCTGCTGCTGTCAGACAAAGAAGAGAAGTTAACAAGAAATAGATTCTGCAGGTAGGGAAAGATATGAGCTATACAGCGGAATTTCCATATCACTCAGATTTTGTTATCTAATTTTTTCCTTCCCACGTCCGCGGGAATCTGTGTATATTACTGCATCTAGATATATGTTATCTTATCTTGGCGCGTACATTTAATTTTCAACGTATTCTATAAGAAATTGCGGGAGTTTTTTTCATGTAGATGATACTGACTGCACGCAAATATAGGCATGATTTATAGGCATGATTTGATGGCTGTACCGATAGGAACGCTAAGAGTAACTTCAGAATCGTTATCCTGGCGGAAAAAATTCATTTGTAAACTTTAAAAAAAAAAGCCAATATCCCCAAAATTATTAAGAGCGCCTCCATTATTAACTAAAATTTCACTCAGCATCCACAATGTATCAGGTATCTACTACAGATATTACATGTGGCGAAAAAGACAAGAACAATGCAATAGCGCATCAAGAAAAAACACAAAGCTTTCAATCAATGAATCGAAAATGTCATTAAAATAGTATATAAATTGAAACTAAGTCATAAAGCTATAAAAAGAAAATTTATTTAAATGCAAGATTTAAAGTAAATTCACTTAGGATCCGTATCTTGGAATTGGGATCAATTTTAAAGGTTTAGTTTTCTGCAAGGCTATACCAAACTTTTCATCCATGTCCAAATCTTTAGGTGAAATGTCTCCCTCTAATTTCCAGTTGAAGAATTGCAATAATGTGGCCAACATCAATGTCAACATCCTTATGGCCAAATTCATACCAGGGCAAATTCTTCTACCAGCACCAAAAGGCAATAAACCAAAATCTCTTCCCTTTACATCAATCTCACAACCTATAAATCTTTCAGGTGAGAAAATATCGGCATTTTGCCAAGCATTTGGATCTCTACCTATGGCCCACAAGTTTACCAAAATTTGAGCATCCTTTGGAACAATGTATCCGTACAATTCTACGTCTGTGTCGGCTTTTCTAGGCAACAAGAAAACTGTAGGTGGGTGCAACCTCAAAGTTTCCTTAATGATTGCTTGCAAGTATGGCAAATTTATTATGTCTGACTCTTGGATTTGCTTATCCTTACCCAATACCTGTTTTATTTCCTCTTGGGCCTTTTCCATCATTTCAGGGTTTCTTATTAACTCAGTCATAACCCACTCAAAGGTAGATGATGTAGTATCAGTACCGGCATCGAAAATATCGACTAACAAGTGGTTAATCTCTCCCATAGTTAATTCATTCTGCTTAAACAATTGCAATAATACGTCCAAGACATCATCTGTAGTTGTAGTAGTTGTAGTTGATGAATCTGGAGCTAACCTCTCACATATTATTCCTTGAAATACTGCAATTAACTTATCAAAAGAACATGCCAACCTTCTTCTAATTCCTGATGGATCGACGCAACCTAAAATTGGGAAGTAATCGGCGTAGTTTGGTTTACCGATGTCTTCCATAATATTCCAAATTAATTCCTTAAACTCTTGAGAGGTATGTGATTTATGATGGGCCAATTCAACAGAAAAGAATAATTTAGACAATAAATTCAAAGAAGTTGTGAAGGCAGCTTTACCAATGTCAACTGCCTGACCCTTCTGAGCGCACTCTTGAACGTATTCATATAATTGCTGAACCTTTGCATGTCTAAAGGTCTGGCATGCATCTAATCTTTGTGGAGACAATAAATGAACAGCAGTTATCTTTCTGAAGTTTCTCCATTTAGGTGAAACAGGCAACCAAGACATAGTCAATTTGTGGTGGTCACCAGCAGTTACTGAATTTGGAATTGTCCTGTTTGATAAAGGGTGATCCTTCTTCAAGAACATCTCTTTGGCGACATCAGCAGATGATACAACAATGGTGGTAACTGATCCTAACCTCAAAGAGATTAAAGGACCGTGGATCTTAGCCAAATTTGCAAATGATCTATGAGGCTTCTTTCCTACTTCCAAGATATTACCAATAATTGGTAAAGGTTTAGGACCTGGAGGTAACAATTTTGTTGTTTGTTGTGAGAATAATAATTTAATAAAATGAAATGAAATAAACCAAATTGCTAATATCATAGCTAATGTTGCATGATCCATTTTTAGTTTATGTATGTGTTTTTTGTAGTTATAGATTTAAGCAAGAAAAGAATACAAACAAAAAATTGAAAAAGATTGATTTAGAATTAAAAAGAAAAATATTTACGTAAGAAGGGAAAATAGTAAATGTTGCAAGTTCACTAAACTCCTAAATTATGCTGCCCTTTATATTCCCTGTTACAGCAGCCGAGCCAAAGGTATATAGGCTCCTTTGCATTAGCATGCGTAACAAACCACCTGTCAGTTTCAACCGAGGTGGTATCCGAGAGAATTGTGTGATTGCTTTAATTAATTTCGGAGAATCTCACATGCCACTGAAGATTAAAAACTGGATGCCAGAAAAGGGGTGTCCAGGTGTAACATCAATAGAGGAAGCTGAAAAGTCTTAGAACGGGTAATCTTCCACCAACCTGATGGGTTCCTAGATATAATCTCGAAGGGAATAAGTAGGGTGATACCGCAGAAGTGTCTGAATGTATTAAGGTCCTCACAGTTTAAATCCCGCTCACACTAACGTAGGATTATTATAACTCAAAAAAATGGCATTATTCTAAGTAAGTTAAATATCCGTAATCTTTAAACAagacttcgTCACCCATGAACCACACGGTTAGTCCAAAAGGGGCAGTTCAGATTCCAGATGCGGGAATTAGCTTGCTGCCACCCTCACCTCACTAACGCTGCGGTGTGCGGATACTTCATGCTATTTATAGACGCGCGTGTCGGAATCAGCACGCGCAAGAACCAAATGGGAAAATCGGAATGGGTCCAGAACTGCTTTGAGTGCTGGCTATTGGCGTCTGATTTCCGTTTTGGGAATCCTTTGCCGCGCGCCCCTCTCAAAACTCCGCACAAGTCCCAGAAAGCGGGAAAGAAATAAAACGCCACCAAAAAAAAAAAAATAAAAGCCAATCCTCGAAGCGTGGGTGGTAGGCCCTGGATTATCCCGTACAAGTATTTCTCAGGAGTAAAAAAACCGTTTGTTTTGGAATTTCCCATTTCGCGGCCACCTACGCCGCTATCTTTGCAACAACTATCTGCGATAACTCAGCAAATTTTGCATATTCGTGTTGCAGTATTGCGATAATGGGAGTCTTACTTCCAACATAACGGCAGAAAGAAATGTGAGAAAATTTTGCATCCTTTGCCTCCGTTCAAGTATATAAAGTCGGCATGCTTGATAATCTTTCTTTCCATCCTACATTGTTCTAATTATTCTTATTCTCCTTTATTCTTTCCTAACATACCAAGAAATTAATCTTCTGTCATTCGCTTAAACACTATATCAATAAAGATCTAAAAATGAAGGGAACCTACTACATCAACCACGGTGACCCATTGATGTACTTGAAGAAGCACATCAAGTTGAGACAATTCTTGGAAGGTTGGCAAGAAAACGTTGTTATCGAAAAGCCAAAGTCTATCTTGATCATCAGTGCACACTGGGACACCAACGTTCCAACCGTTAACTTCGTTGAACACTGTGACACCATCCACGACTTCGACGACTACCCAGACCCATTGTACCAAATCCAATACCGCGCTCCAGGTGCTCCAAACTTGGCTAAGAAGGTTGAAGAATTGCTTAAGGAATCTGGTATGGAATGTGAAATCGACACCAAGAGAGGTTTGGACCACGCTGCTTGGTTCCCATTGATGTTCATGTACCCAGAAGCTAACATCCCAATCTGTGAATTGTCTGTTCAACCATCTAAGGACGGTATCCACCACTACAACGTTGGTAAAGCTTTGTCTCCATTGTTGCAACAAGGTGTTTTGATCATCGGTTCTGGTGGAACCGTTCACCCATCTGACGACACCCCACACTGTCCAAACGGTGTTGCTCCATGGGCTATCGAGTTCGACAACTGGTTGGAAGACGCTTTGCTCTCGGGTAGATACGAAGACGTTAACAACTTCAAGAAGTTGGCTCCAAACTGGGAAATCTCTCACCCAGGTCAAGAACACTTGTACCCATTGCACGTTGCTTTGGGTGCTGCTGGTAAGAACCCAAAGACCCAATTGATCCACCGGTCTTGGGCTGCTAACGGTGTTTTCGGTTACTCTACCTACAACTTCACCCCAACCACCCAAAAGACCGACGGATCCTAAATCGCGTGCATTCATCCGCTCTAACCGAAAAGGAAGGAGTTAGACAACCTGAAGTCTAGGTCCCTATTTATTTTTTTATAGTTATGTTAGTATTAAGAACGTTATTTATATTTCAAATTTTTCTTTTTTTTCTGTACAGACGCGTGTACGCATGTAACATTATACTGAAAACCTTGCTTGAGAAGGTTTTGGGACGCTCGAAGATCGCGTCAGCTGAAGCTTCGTACGCTGCAGGTCGACAATTCTGCAGGTCGACAACCCTTAATATAACTTCGTATAATGTATGCTATACGAAGTTATTAGGTCTAGAGATCTGTTTAGCTTGCCTCGTCCCCGCCGGGTCACCCGGCCAGCGACATGGAGGCCCAGAATACCCTCCTTGACAGTCTTGACGTGCGCAGCTCAGGGGCATGATGTGACTGTCGCCCGTACATTTAGCCCATACATCCCCATGTATAATCATTTGCATCCATACATTTTGATGGCCGCACGGCGCGAAGCAAAAATTACGGCTCCTCGCTGCAGACCTGCGAGCAGGGAAACGCTCCCCTCACAGACGCGTTGAATTGTCCCCACGCCGCGCCCCTGTAGAGAAATATAAAAGGTTAGGATTTGCCACTGAGGTTCTTCTTTCATATACTTCCTTTTAAAATCTTGCTAGGATACAGTTCTCACATCACATCCGAACATAAACAACCATGGGTACCACTCTTGACGACACGGCTTACCGGTACCGCACCAGTGTCCCGGGGGACGCCGAGGCCATCGAGGCACTGGATGGGTCCTTCACCACCGACACCGTCTTCCGCGTCACCGCCACCGGGGACGGCTTCACCCTGCGGGAGGTGCCGGTGGACCCGCCCCTGACCAAGGTGTTCCCCGACGACGAATCGGACGACGAATCGGACGACGGGGAGGACGGCGACCCGGACTCCCGGACGTTCGTCGCGTACGGGGACGACGGCGACCTGGCGGGCTTCGTGGTCGTCTCGTACTCCGGCTGGAACCGCCGGCTGACCGTCGAGGACATCGAGGTCGCCCCGGAGCACCGGGGGCACGGGGTCGGGCGCGCGTTGATGGGGCTCGCGACGGAGTTCGCCCGCGAGCGGGGCGCCGGGCACCTCTGGCTGGAGGTCACCAACGTCAACGCACCGGCGATCCACGCGTACCGGCGGATGGGGTTCACCCTCTGCGGCCTGGACACCGCCCTGTACGACGGCACCGCCTCGGACGGCGAGCAGGCGCTCTACATGAGCATGCCCTGCCCCTAATCAGTACTGACAATAAAAAGATTCTTGTTTTCAAGAACTTGTCATTTGTATAGTTTTTTTATATTGTAGTTGTTCTATTTTAATCAAATGTTAGCGTGATTTATATTTTTTTTCGCCTCGACATCATCTGCCCAGATGCGAAGTTAAGTGCGCAGAAAGTAATATCATGCGTCAATCGTATGTGAATGCTGGTCGCTATACTGCTGTCGATTCGATACTAACGCCGCCATCCAGTGTCGAAAACGAGCTCTCGAGAACCCTTAAT'
BTX['need_reverse']=0 #if parts needs to be reversed:1 else:0
BTX['Primer_up_U']='GV1' #user site up
BTX['Primer_down_U']='GV2'#user site down
BTX['Primer_up']=''
BTX['Primer_up_Tm']=''
BTX['Primer_down']=''
BTX['Primer_down_Tm']=''


## CDSs ups and downs regions will be linked to KanMx to generate KO plasmids
KanMX={}
KanMX['name']='pTEF::KanMX::tTef'
KanMX['seq']='TAGGTCTAGAGATCTGTTTAGCttgcctcgtccccgccgggtcacccggccagcgacatggaggcccagaataccctccttgacagtcttgacgtgcgcagctcaggggcatgatgtgactgtcgcccgtacatttagcccatacatccccatgtataatcatttgcatccatacattttgatggccgcacggcgcgaagcaaaaattacggctcctcgctgcagacctgcgagcagggaaacgctcccctcacagacgcgttgaattgtccccacgccgcgcccctgtagagaaatataaaaggttaggatttgccactgaggttcttctttcatatacttccttttaaaatcttgctaggatacagttctcacatcacatccgaacataaacaaccatgggtaaggaaaagactcacgtttcgaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccggcaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataagcttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaatcagtactgacaataaaaagattcttgttttcaagaacttgtcatttgtatagtttttttatattgtagttgttctattttaatcaaatgttagcgtgatttatattttttttcgcctcgacatcatctgcccagatgcgaagttaagtgcgcagaaagtaatatcatgcgtcaatcgtatgtgaatgctggtcgctatactgctgtcgattcgatactaacgccgccatccagtgtcgaaaacgaGCTCTCGAGAACCCTTAAT'
KanMX['need_reverse']=0 #if parts needs to be reversed:1 else:0
KanMX['Primer_up_U']='U1' #user site up
KanMX['Primer_down_U']='U2'#user site down
KanMX['Primer_up']=''
KanMX['Primer_up_Tm']=''
KanMX['Primer_down']=''
KanMX['Primer_down_Tm']=''



##in addition,CDS will be PCRed with the forward primer of the Up and the reverse of the down
#and annealed into pCCM022 to be flanked by  HomoB and XII-dwn

KGD2_up={}
KGD2_up['name']='KGD2_up'
KGD2_up['seq']='ggcaggaccttagttttgattggctgcaaaaaaagggttgttaatatcgtctttgacggagatgcgggtggctggaattgttttgtttattattccccaagagagaagagacaagaagtagtaatgacgatggaacaaagatcggttctaataggaccacagggttgaatagagacgagtaaagacgcaacggtcaaggctttccacgagacgttcgaat'
KGD2_up['need_reverse']=1 #if parts needs to be reversed:1 else:0
KGD2_up['Primer_up_U']='GV1' #user site up
KGD2_up['Primer_down_U']='U1'#user site down
KGD2_up['Primer_up']=''
KGD2_up['Primer_up_Tm']=''
KGD2_up['Primer_down']=''
KGD2_up['Primer_down_Tm']=''

KGD2_dw={}
KGD2_dw['name']='KGD2_dw'
KGD2_dw['seq']='atgtggaaatcaaaagaatattagttgattctattatataataaatagttataatcattgtattttgatgatgaaaattcagtatctaagagttatagccaagagagtaatttttaatacttttttgcgcttttccatatctcattataaaaagaaaacattaattcatagttccatttatcatcatagcatatctttaa'
KGD2_dw['need_reverse']=1 #if parts needs to be reversed:1 else:0
KGD2_dw['Primer_up_U']='U2' #user site up
KGD2_dw['Primer_down_U']='GV2'#user site down
KGD2_dw['Primer_up']=''
KGD2_dw['Primer_up_Tm']=''
KGD2_dw['Primer_down']=''
KGD2_dw['Primer_down_Tm']=''

KGD2=[KGD2_up,KanMX,KGD2_dw]

HEM25_up={}
HEM25_up['name']='HEM25_up'
HEM25_up['seq']='atttcacttcgtccttaatattttaacacattaactgtttgctataagcatacattgcttttaccactttttttaataatatctttttatccgtattaattaccgcatataattttagttcgaaacattaaaggtcagtcccgaagttataattataacatttaattcctgaatttaccgtattacatttaaatttgcat'
HEM25_up['need_reverse']=1 #if parts needs to be reversed:1 else:0
HEM25_up['Primer_up_U']='GV1' #user site up
HEM25_up['Primer_down_U']='U1'#user site down
HEM25_up['Primer_up']=''
HEM25_up['Primer_up_Tm']=''
HEM25_up['Primer_down']=''
HEM25_up['Primer_down_Tm']=''

HEM25_dw={}
HEM25_dw['name']='HEM25_dw'
HEM25_dw['seq']='gtgcacttatccatcgactctcttatatttatttaaacacttgtaacgcatccaatctatttcatagttagagtctctcttccttgttattgatatgatttttcttttgtaccgtgtgtgtatgtatgtatgtatgaacgtactactaagcttcaacgtcactctataaagaaactaaaaagacacgtatttatacctt'
HEM25_dw['need_reverse']=1 #if parts needs to be reversed:1 else:0
HEM25_dw['Primer_up_U']='U2' #user site up
HEM25_dw['Primer_down_U']='GV2'#user site down
HEM25_dw['Primer_up']=''
HEM25_dw['Primer_up_Tm']=''
HEM25_dw['Primer_down']=''
HEM25_dw['Primer_down_Tm']=''


HEM25=[HEM25_up,KanMX,HEM25_dw]



VMA16_up={} #PPA1
VMA16_up['name']='VMA16_up'
VMA16_up['seq']='ggcctattgggtatctgcttgtcaggtgcaggcccaactatcttggctttggccactgagaatttcgaagaaatctctcaagaaattatcaacaggttcgccaaaaacggcatcaagtgctcctggaaactactggagcctgcctacgatggtgctagcgtcgaacagcaatgaaatgtcctttacccctagtatactgtgtt'
VMA16_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
VMA16_up['Primer_up_U']='GV1' #user site up
VMA16_up['Primer_down_U']='U1'#user site down
VMA16_up['Primer_up']=''
VMA16_up['Primer_up_Tm']=''
VMA16_up['Primer_down']=''
VMA16_up['Primer_down_Tm']=''

VMA16_dw={}
VMA16_dw['name']='VMA16_dw'
VMA16_dw['seq']='agcgcatccatatttactaatagatataatgagatatctgagatacatgtgtatgtatatatgcacgttttcttttattatctaaaaatcatattatattaagtaagagaaaaaaatgtacaactatataaatatatatttatttaaaatggttttgaatttttcctattctggttgatattgcccaaaagctattcagt'
VMA16_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
VMA16_dw['Primer_up_U']='U2' #user site up
VMA16_dw['Primer_down_U']='GV2'#user site down
VMA16_dw['Primer_up']=''
VMA16_dw['Primer_up_Tm']=''
VMA16_dw['Primer_down']=''
VMA16_dw['Primer_down_Tm']=''
VMA16 = [VMA16_up,KanMX,VMA16_dw]




CAX4_up={} 
CAX4_up['name']='CAX4_up'
CAX4_up['seq']='aaccccgagaatttaaatagaacagagtatatgcatttattaacagttattacgccttgtccccagggagagtagtaagggatatgggcatttataaatgtacataagtcacgtttttgcagccaataaaatataataatagaatagtaacaaggaaagctaggccaaaactccttacatggagctagtatacccctttt'
CAX4_up['need_reverse']=1 #if parts needs to be reversed:1 else:0
CAX4_up['Primer_up_U']='GV1' #user site up
CAX4_up['Primer_down_U']='U1'#user site down
CAX4_up['Primer_up']=''
CAX4_up['Primer_up_Tm']=''
CAX4_up['Primer_down']=''
CAX4_up['Primer_down_Tm']=''

CAX4_dw={}
CAX4_dw['name']='CAX4_dw'
CAX4_dw['seq']='aagtgtgggtgacatgctaaatccgcggctttccttaagctttttcacaaaccatctattatgtccagaagctccgctgattatccgtggaaaagggtcttattctgccgtaggaaacttcttctcgagaaaaaagtgagttcacccgcggcgctctcgtaaaacccgagggccgattctttccctccgtggaacaatcg'
CAX4_dw['need_reverse']=1 #if parts needs to be reversed:1 else:0
CAX4_dw['Primer_up_U']='U2' #user site up
CAX4_dw['Primer_down_U']='GV2'#user site down
CAX4_dw['Primer_up']=''
CAX4_dw['Primer_up_Tm']=''
CAX4_dw['Primer_down']=''
CAX4_dw['Primer_down_Tm']=''
CAX4 = [CAX4_up,KanMX,CAX4_dw]


PRO2_up={} #PPA1
PRO2_up['name']='PRO2_up'
PRO2_up['seq']='ttgtaaagtgaccttgttttgactttttaacgagtaaaaaaacgagttaacaaattcttgacgtaaacgataaagccgcccatcaaaagaaatgcaataatgtcaatgatgaaatacctatacaggtagatggatttcgtgttgttcttcgatttatttaacacgtttttcagctctatcaaactatccatttcccttag'
PRO2_up['need_reverse']=1 #if parts needs to be reversed:1 else:0
PRO2_up['Primer_up_U']='GV1' #user site up
PRO2_up['Primer_down_U']='U1'#user site down
PRO2_up['Primer_up']=''
PRO2_up['Primer_up_Tm']=''
PRO2_up['Primer_down']=''
PRO2_up['Primer_down_Tm']=''

PRO2_dw={}
PRO2_dw['name']='PRO2_dw'
PRO2_dw['seq']='gagtagatgtaagacgtgaaaatgccatggtaaaacgtattgtttagtgaaatgtatactaagtaggagtaaaaatcagcaatttctggtgttgaaatgtgaacaacagcgttctcctccgaaaaagcaagctataacgttgctcaacttttgattccgaaatcagaagcttaaataaggaaaactgaaagctaattgga'
PRO2_dw['need_reverse']=1 #if parts needs to be reversed:1 else:0
PRO2_dw['Primer_up_U']='U2' #user site up
PRO2_dw['Primer_down_U']='GV2'#user site down
PRO2_dw['Primer_up']=''
PRO2_dw['Primer_up_Tm']=''
PRO2_dw['Primer_down']=''
PRO2_dw['Primer_down_Tm']=''
PRO2 = [PRO2_up,KanMX,PRO2_dw]


GLR1_up={} #PPA1
GLR1_up['name']='GLR1_up'
GLR1_up['seq']='tggtcaattctatgccttttatgcacattgcatgagtattctaaaaagatgctgcatgctgcccgtaaatcttcattattttcagagtcaggtacggaaaagacgacagtttctccgtataacagcattgaaagcgtggaagaatcaaactcggctctagattttacgcgtttagcataaaattgttggttttatacgta'
GLR1_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
GLR1_up['Primer_up_U']='GV1' #user site up
GLR1_up['Primer_down_U']='U1'#user site down
GLR1_up['Primer_up']=''
GLR1_up['Primer_up_Tm']=''
GLR1_up['Primer_down']=''
GLR1_up['Primer_down_Tm']=''

GLR1_dw={}
GLR1_dw['name']='GLR1_dw'
GLR1_dw['seq']='cttttactgctttctaactttaagccttcaaagaagaagctcttctctttctgatttcagccttctcagcctttctttcagacaatctcttagccaacaattgagcgtattcggcagcagcttctctttgagcttgagcgtttctgaccttcaaagctctttggtgtctctttctttgcaatctttgaggagtaaccaatc'
GLR1_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
GLR1_dw['Primer_up_U']='U2' #user site up
GLR1_dw['Primer_down_U']='GV2'#user site down
GLR1_dw['Primer_up']=''
GLR1_dw['Primer_up_Tm']=''
GLR1_dw['Primer_down']=''
GLR1_dw['Primer_down_Tm']=''
GLR1 = [GLR1_up,KanMX,GLR1_dw]


YSY6_up={} 
YSY6_up['name']='YSY6_up'
YSY6_up['seq']='ggagtcttcttgcgagaaaagaaataattataatataaaaaaaaattgaatggagataagaagtttaactagcaaagttcgagagatataaaaagtagagatgcgaaaataaaattaaaacaaatctgaaaataatggaagtgaaacaaggctaagcaagaaaattttctatattgccaagacaggaacgtttgtgatga'
YSY6_up['need_reverse']=0 #if parts needs to be reversed:1 else:0
YSY6_up['Primer_up_U']='GV1' #user site up
YSY6_up['Primer_down_U']='U1'#user site down
YSY6_up['Primer_up']=''
YSY6_up['Primer_up_Tm']=''
YSY6_up['Primer_down']=''
YSY6_up['Primer_down_Tm']=''

YSY6_dw={}
YSY6_dw['name']='YSY6_dw'
YSY6_dw['seq']='gcagtattacccgcgttgtcatttatggatgcgaaaaaagatgaagtaattcaagtgctattgtgatttaccatgccaatcaaacgtgcaacaaagatgagtatgaggggggacccaggcatagtacacagaaaagctttgtagactgtctttgttttcttaatttttcaagtaggctttctgcttttcttctgtactta'
YSY6_dw['need_reverse']=0 #if parts needs to be reversed:1 else:0
YSY6_dw['Primer_up_U']='U2' #user site up
YSY6_dw['Primer_down_U']='GV2'#user site down
YSY6_dw['Primer_up']=''
YSY6_dw['Primer_up_Tm']=''
YSY6_dw['Primer_down']=''
YSY6_dw['Primer_down_Tm']=''
YSY6 = [YSY6_up,KanMX,YSY6_dw]




PDX3_up={} 
PDX3_up['name']='PDX3_up'
PDX3_up['seq']='aggtagagtatctttaaaatatctaatcatgcaaattagtcatttcttctttcttataagttgcggccccttctagtatttacaatttacccggcgctggttctggtggcatctataagagctctatctggctgctaagcagacttgctgttacttgtatgtgctgtttatcaagattggggcattacgcatgtgtatgt'
PDX3_up['need_reverse']=1 #if parts needs to be reversed:1 else:0
PDX3_up['Primer_up_U']='GV1' #user site up
PDX3_up['Primer_down_U']='U1'#user site down
PDX3_up['Primer_up']=''
PDX3_up['Primer_up_Tm']=''
PDX3_up['Primer_down']=''
PDX3_up['Primer_down_Tm']=''

PDX3_dw={}
PDX3_dw['name']='PDX3_dw'
PDX3_dw['seq']='tctttggatgaatttgtcggttctaactcttttttggaaacggcaaggtgagcagtaggcagctcatctcttcaactatttttataacatataatttccctttgtgaaaaaattttcccgccatttccaaatttcggaattcctttttcttttttttcatctcggtatattctgacatattaccctataggcggtataaa'
PDX3_dw['need_reverse']= 1 #if parts needs to be reversed:1 else:0
PDX3_dw['Primer_up_U']='U2' #user site up
PDX3_dw['Primer_down_U']='GV2'#user site down
PDX3_dw['Primer_up']=''
PDX3_dw['Primer_up_Tm']=''
PDX3_dw['Primer_down']=''
PDX3_dw['Primer_down_Tm']=''
PDX3 = [PDX3_up,KanMX,PDX3_dw]

In [10]:
UpXII5={}
UpXII5['name']='UpXII5'
UpXII5['seq']='GGGGGGGGGGGGGGGGGGGGGGGGGGGccggaggaggccgctataaccggtttgaatttattgtcacagtgtcacatcagc'
UpXII5['need_reverse']= 0 #if parts needs to be reversed:1 else:0
UpXII5['Primer_up_U']='U2' #user site up
UpXII5['Primer_down_U']='GV1'#user site down
UpXII5['Primer_up']=''
UpXII5['Primer_up_Tm']=''
UpXII5['Primer_down']=''
UpXII5['Primer_down_Tm']=''

HomoB={}
HomoB['name']='HomoB'
HomoB['seq']='ACTGAATACAGGCTAGTGAATGCTCTCGGTCTTGTACGTGTGCGATTGACGGGGGGGGGGGGGGGGGGGGGGGGGGG'
HomoB['need_reverse']= 0 #if parts needs to be reversed:1 else:0
HomoB['Primer_up_U']='GV1' #user site up
HomoB['Primer_down_U']='U2'#user site down
HomoB['Primer_up']=''
HomoB['Primer_up_Tm']=''
HomoB['Primer_down']=''
HomoB['Primer_down_Tm']=''

HomoA={}
HomoA['name']='HomoA'
HomoA['seq']='GGGGGGGGGGGGGGGGGGGGGGGGGGGGTACTCGGCTATGAGATTGAATACTCACGAAGGTAACGGTGGCACGACAAC'
HomoA['need_reverse']= 0 #if parts needs to be reversed:1 else:0
HomoA['Primer_up_U']='GV1' #user site up
HomoA['Primer_down_U']='U2'#user site down
HomoA['Primer_up']=''
HomoA['Primer_up_Tm']=''
HomoA['Primer_down']=''
HomoA['Primer_down_Tm']=''


DwXII5={}
DwXII5['name']='DwXII5'
DwXII5['seq']='gcaactcagaagtttgacagcaagcaagttcatcattcgaactagccttattgttttagttcGGGGGGGGGGGGGGGGGGGGGGGGGGGG'
DwXII5['need_reverse']= 0 #if parts needs to be reversed:1 else:0
DwXII5['Primer_up_U']='GV1' #user site up
DwXII5['Primer_down_U']='U2'#user site down
DwXII5['Primer_up']=''
DwXII5['Primer_up_Tm']=''
DwXII5['Primer_down']=''
DwXII5['Primer_down_Tm']=''



BB_PCR = [UpXII5,DwXII5,HomoA,HomoB]

In [12]:
Parts = dt.User_Assebmly(BB_PCR, 50, User_Overhangs)
print(Parts)

Will use input Tm_target for design:  50
[{'name': 'UpXII5', 'seq': 'GGGGGGGGGGGGGGGGGGGGGGGGGGGCCGGAGGAGGCCGCTATAACCGGTTTGAATTTATTGTCACAGTGTCACATCAGC', 'need_reverse': 0, 'Primer_up_U': 'U2', 'Primer_down_U': 'GV1', 'Primer_up': 'ACACAGGCuGGGGGGGGGGG', 'Primer_up_Tm': 55.40352299998773, 'Primer_down': 'ATCGCACuGCTGATGTGACACTGTGAC', 'Primer_down_Tm': 50.61273634503215}, {'name': 'DwXII5', 'seq': 'GCAACTCAGAAGTTTGACAGCAAGCAAGTTCATCATTCGAACTAGCCTTATTGTTTTAGTTCGGGGGGGGGGGGGGGGGGGGGGGGGGGG', 'need_reverse': 0, 'Primer_up_U': 'GV1', 'Primer_down_U': 'U2', 'Primer_up': 'CGTGCGAuGCAACTCAGAAGTTTGACAGC', 'Primer_up_Tm': 52.21562927265825, 'Primer_down': 'AGCCTGTGuCCCCCCCCCCC', 'Primer_down_Tm': 55.40352299998773}, {'name': 'HomoA', 'seq': 'GGGGGGGGGGGGGGGGGGGGGGGGGGGGTACTCGGCTATGAGATTGAATACTCACGAAGGTAACGGTGGCACGACAAC', 'need_reverse': 0, 'Primer_up_U': 'GV1', 'Primer_down_U': 'U2', 'Primer_up': 'CGTGCGAuGGGGGGGGGGG', 'Primer_up_Tm': 55.40352299998773, 'Primer_down': 'AGCCTGTGuGTTGTCGTGCCACCG', 

### Design

In [13]:
importlib.reload(dt)
Parts = dt.User_Assebmly(YER084W_KO_BTX, 50, User_Overhangs)
print(Parts)

NameError: name 'YER084W_KO_BTX' is not defined

#### compile into a table

In [14]:



primer_name=[]
primer_dir=[]
primer_tm=[]
primer_OH=[]
primer_template_name=[]
primer_seq=[]
primer_template_len=[]
benchling=[]
i=108
for part in Parts:
    #here we adapt the order with which the overhang names are appended to match the Overhang names to the inversion of sequences
    if part['need_reverse']==0:
        primer_OH.append(part['Primer_up' +'_U'])
        primer_OH.append(part['Primer_down'+'_U'])
    elif part['need_reverse']==1:
        primer_OH.append(part['Primer_down'+'_U'])
        primer_OH.append(part['Primer_up' +'_U'])
        
    for primer in ['Primer_up', 'Primer_down']:
        primer_name.append('BTX.1')#.zfill fills primer number wiht leading 0
        primer_seq.append(part[primer])
        primer_tm.append(round(part[primer +'_Tm'],2))
        primer_template_len.append(len(part['seq']))
        primer_template_name.append(part['name'])
        #primer_OH.append(part[primer +'_U'])
        primer_dir.append('fwd'  if primer[7:] == 'up' else 'rvs' )
        benchling.append(part[primer].replace('u','t'))
        i+=1
## need to fix the current overhang name bug (if seq needs inversion the OH name doenst match)
        


df = np.stack([primer_name, primer_seq,primer_dir, primer_tm,primer_OH, primer_template_name, primer_template_len, benchling] ).T
Table=pd.DataFrame(df , columns=['Primer Name', 'Seq','Dir', 'Tm (c)','User', 'Template', 'Template_Len','benchling'])

Table.set_index('Primer Name')

,Seq,Dir,Tm (c),User,Template,Template_Len,benchling
Primer Name,,,,,,,
BTX.1,ACACAGGCuGGGGGGGGGGG,fwd,55.4,U2,UpXII5,81,ACACAGGCtGGGGGGGGGGG
BTX.1,ATCGCACuGCTGATGTGACACTGTGAC,rvs,50.61,GV1,UpXII5,81,ATCGCACtGCTGATGTGACACTGTGAC
BTX.1,CGTGCGAuGCAACTCAGAAGTTTGACAGC,fwd,52.22,GV1,DwXII5,90,CGTGCGAtGCAACTCAGAAGTTTGACAGC
BTX.1,AGCCTGTGuCCCCCCCCCCC,rvs,55.4,U2,DwXII5,90,AGCCTGTGtCCCCCCCCCCC
BTX.1,CGTGCGAuGGGGGGGGGGG,fwd,55.4,GV1,HomoA,78,CGTGCGAtGGGGGGGGGGG
BTX.1,AGCCTGTGuGTTGTCGTGCCACCG,rvs,51.13,U2,HomoA,78,AGCCTGTGtGTTGTCGTGCCACCG
BTX.1,CGTGCGAuACTGAATACAGGCTAGTGAATG,fwd,51.39,GV1,HomoB,77,CGTGCGAtACTGAATACAGGCTAGTGAATG
BTX.1,AGCCTGTGuCCCCCCCCCCC,rvs,55.4,U2,HomoB,77,AGCCTGTGtCCCCCCCCCCC


In [77]:
for i in range(112,112+32):
    print('BTX.'+str(i))

BTX.112
BTX.113
BTX.114
BTX.115
BTX.116
BTX.117
BTX.118
BTX.119
BTX.120
BTX.121
BTX.122
BTX.123
BTX.124
BTX.125
BTX.126
BTX.127
BTX.128
BTX.129
BTX.130
BTX.131
BTX.132
BTX.133
BTX.134
BTX.135
BTX.136
BTX.137
BTX.138
BTX.139
BTX.140
BTX.141
BTX.142
BTX.143


### #design primers for simple insertion in User plasmid

In [60]:


KanMX={}
KanMX['name']='pTEF::KanMX::tTef'
KanMX['seq']='TAGGTCTAGAGATCTGTTTAGCttgcctcgtccccgccgggtcacccggccagcgacatggaggcccagaataccctccttgacagtcttgacgtgcgcagctcaggggcatgatgtgactgtcgcccgtacatttagcccatacatccccatgtataatcatttgcatccatacattttgatggccgcacggcgcgaagcaaaaattacggctcctcgctgcagacctgcgagcagggaaacgctcccctcacagacgcgttgaattgtccccacgccgcgcccctgtagagaaatataaaaggttaggatttgccactgaggttcttctttcatatacttccttttaaaatcttgctaggatacagttctcacatcacatccgaacataaacaaccatgggtaaggaaaagactcacgtttcgaggccgcgattaaattccaacatggatgctgatttatatgggtataaatgggctcgcgataatgtcgggcaatcaggtgcgacaatctatcgattgtatgggaagcccgatgcgccagagttgtttctgaaacatggcaaaggtagcgttgccaatgatgttacagatgagatggtcagactaaactggctgacggaatttatgcctcttccgaccatcaagcattttatccgtactcctgatgatgcatggttactcaccactgcgatccccggcaaaacagcattccaggtattagaagaatatcctgattcaggtgaaaatattgttgatgcgctggcagtgttcctgcgccggttgcattcgattcctgtttgtaattgtccttttaacagcgatcgcgtatttcgtctcgctcaggcgcaatcacgaatgaataacggtttggttgatgcgagtgattttgatgacgagcgtaatggctggcctgttgaacaagtctggaaagaaatgcataagcttttgccattctcaccggattcagtcgtcactcatggtgatttctcacttgataaccttatttttgacgaggggaaattaataggttgtattgatgttggacgagtcggaatcgcagaccgataccaggatcttgccatcctatggaactgcctcggtgagttttctccttcattacagaaacggctttttcaaaaatatggtattgataatcctgatatgaataaattgcagtttcatttgatgctcgatgagtttttctaatcagtactgacaataaaaagattcttgttttcaagaacttgtcatttgtatagtttttttatattgtagttgttctattttaatcaaatgttagcgtgatttatattttttttcgcctcgacatcatctgcccagatgcgaagttaagtgcgcagaaagtaatatcatgcgtcaatcgtatgtgaatgctggtcgctatactgctgtcgattcgatactaacgccgccatccagtgtcgaaaacgaGCTCTCGAGAACCCTTAAT'
KanMX['need_reverse']=0 #if parts needs to be reversed:1 else:0
KanMX['Primer_up_U']='GV1' #user site up
KanMX['Primer_down_U']='GV2'#user site down
KanMX['Primer_up']=''
KanMX['Primer_up_Tm']=''
KanMX['Primer_down']=''
KanMX['Primer_down_Tm']=''

#Design:
importlib.reload(dt)
Parts = dt.User_Assebmly([KanMX], 50, User_Overhangs)
print(Parts)




primer_name=[]
primer_dir=[]
primer_tm=[]
primer_OH=[]
primer_template_name=[]
primer_seq=[]
primer_template_len=[]
benchling=[]
i=1
for part in Parts:
    #here we adapt the order with which the overhang names are appended to match the Overhang names to the inversion of sequences
    if part['need_reverse']==0:
        primer_OH.append(part['Primer_up' +'_U'])
        primer_OH.append(part['Primer_down'+'_U'])
    elif part['need_reverse']==1:
        primer_OH.append(part['Primer_down'+'_U'])
        primer_OH.append(part['Primer_up' +'_U'])
        
    for primer in ['Primer_up', 'Primer_down']:
        primer_name.append('XX.0')#.zfill fills primer number wiht leading 0
        primer_seq.append(part[primer])
        primer_tm.append(round(part[primer +'_Tm'],2))
        primer_template_len.append(len(part['seq']))
        primer_template_name.append(part['name'])
        #primer_OH.append(part[primer +'_U'])
        primer_dir.append('fwd'  if primer[7:] == 'up' else 'rvs' )
        benchling.append(part[primer].replace('u','t'))
        i+=1
## need to fix the current overhang name bug (if seq needs inversion the OH name doenst match)
        


df = np.stack([primer_name, primer_seq,primer_dir, primer_tm,primer_OH, primer_template_name, primer_template_len, benchling] ).T
Table=pd.DataFrame(df , columns=['Primer Name', 'Seq','Dir', 'Tm (c)','User', 'Template', 'Template_Len','benchling'])

Table.set_index('Primer Name')

Will use input Tm_target for design:  50
[{'name': 'pTEF::KanMX::tTef', 'seq': 'TAGGTCTAGAGATCTGTTTAGCTTGCCTCGTCCCCGCCGGGTCACCCGGCCAGCGACATGGAGGCCCAGAATACCCTCCTTGACAGTCTTGACGTGCGCAGCTCAGGGGCATGATGTGACTGTCGCCCGTACATTTAGCCCATACATCCCCATGTATAATCATTTGCATCCATACATTTTGATGGCCGCACGGCGCGAAGCAAAAATTACGGCTCCTCGCTGCAGACCTGCGAGCAGGGAAACGCTCCCCTCACAGACGCGTTGAATTGTCCCCACGCCGCGCCCCTGTAGAGAAATATAAAAGGTTAGGATTTGCCACTGAGGTTCTTCTTTCATATACTTCCTTTTAAAATCTTGCTAGGATACAGTTCTCACATCACATCCGAACATAAACAACCATGGGTAAGGAAAAGACTCACGTTTCGAGGCCGCGATTAAATTCCAACATGGATGCTGATTTATATGGGTATAAATGGGCTCGCGATAATGTCGGGCAATCAGGTGCGACAATCTATCGATTGTATGGGAAGCCCGATGCGCCAGAGTTGTTTCTGAAACATGGCAAAGGTAGCGTTGCCAATGATGTTACAGATGAGATGGTCAGACTAAACTGGCTGACGGAATTTATGCCTCTTCCGACCATCAAGCATTTTATCCGTACTCCTGATGATGCATGGTTACTCACCACTGCGATCCCCGGCAAAACAGCATTCCAGGTATTAGAAGAATATCCTGATTCAGGTGAAAATATTGTTGATGCGCTGGCAGTGTTCCTGCGCCGGTTGCATTCGATTCCTGTTTGTAATTGTCCTTTTAACAGCGATCGCGTATTTCGTCTCGCTCAGGCGCAATCACGAATGAATAACGGTTTGGTTGATGCGAGTGATTTTGATGACGAGCGTAATGGCTGGCCTGTTGAA

,Seq,Dir,Tm (c),User,Template,Template_Len,benchling
Primer Name,,,,,,,
XX.0,CGTGCGAuTAGGTCTAGAGATCTGTTTAGC,fwd,50.27,GV1,pTEF::KanMX::tTef,1473,CGTGCGAtTAGGTCTAGAGATCTGTTTAGC
XX.0,CACGCGAuATTAAGGGTTCTCGAGAGC,rvs,50.05,GV2,pTEF::KanMX::tTef,1473,CACGCGAtATTAAGGGTTCTCGAGAGC
